# Covariance matrix change-point detection under graph stationarity assumption


## Problem formulation

Let $y = (y_1, \ldots, y_t, \ldots, y_T), y_t \in \mathbb{R}^{N}$ a graph signal lying on the nodes of the graph $G = (V, E, W)$, with $N =|V|$.

We aim at detecting changes of the (spatial) covariance matrix $\Sigma_t$ of the graph signals $y_t$. We assume that there exits an unknown set of change-points $\Tau = (t_1, \ldots, t_K) \subset [1, T]$ with unknown cardinality such that the covariance matrix of the graph signals is constant over any segment $[t_k, t_{k+1}]$. We do the following hypothesis:

1. the signals $y_t$ follow a multivariate Gaussian distribution with fixed covariance matrix over each segment and known mean $\mu$, i.e:
$$\forall k \in [1, K] ~ \forall t \in [t_k, t_{k+1}] \quad y_t \sim \mathcal{N}(\mu, \Sigma_k)$$

2. over each segment, the signals $y_t$ verify the second order wide-sense graph stationarity:
$$\forall k \in [1, K] \quad \Sigma_k = U \text{diag}(\gamma_k)U^T $$

where the matrix $U$ contains the eigenvectors of the graph combinatorial Laplacian matrix $L = D - W$ in its columns. 

The Graph Fourier Transform $\tilde{y}$ of a signal $y$ is defined by $\tilde{y} = U^T y $.


Based on the above assumptions, the cost derived from the maximum log-likelihood over a segment $[a, b-1]$ writes:

\begin{align*}
    c_s(y_{a}, \ldots, y_{b-1}) = ~ & (b - a) \sum_{n=1}^N \log \hat{\gamma}_{a.b}^{(n)} ~ + ~ \sum_{t=a}^{b-1} \sum_{n=1}^N \frac{\left(\tilde{y}_t^{(n)} - \hat{\tilde{\mu}}_T^{(n)}\right)^2}{\hat{\gamma}_{a.b}^{(n)}} = ~ (b - a) \sum_{n=1}^N \log \hat{\gamma}_{a.b}^{(n)} ~ + N(b-a)
\end{align*}

where:

- $\hat{\mu}_{T}$ is the empirical mean of the process over $[0, T]$
- $\hat{\gamma}_{a..b}$ is the (empirical) biased correlogram/periodogram of the process over $[a, b-1]$: $\hat{\gamma}_{a..b} = \frac{1}{(b-a)} \sum_{t=a}^{b-1} \left(\tilde{y}_t^{(n)} - \hat{\tilde{\mu}}_T^{(n)}\right)^2$

### Computation of the cost

In [ ]:
import time
import subprocess
import json
import os
import warnings

import numpy as np
import networkx as nx
import ruptures as rpt
import matplotlib.pyplot as plt

from scipy.linalg import eigh
from ruptures.base import BaseCost
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
from numba import njit
from typing import List, Callable, Literal, Dict
from scipy.spatial.distance import pdist, squareform
from matplotlib.lines import Line2D


In [ ]:
class CostGraphStatioNormal(BaseCost):

    """
    """

    model = "graph_sationary_normal_cost"

    def __init__(self, laplacian_mat) -> None:
        """
        Args:
            laplacian_mat (array): the discrete Laplacian matrix of the graph: D - W
            where D is the diagonal matrix diag(d_i) of the node degrees and W the adjacency matrix
        """
        self.graph_laplacian_mat = laplacian_mat
        self.signal = None
        self.gft_square_cumsum = None
        self.gft_mean = None
        self.min_size = laplacian_mat.shape[0]
        super().__init__()
    
    def fit(self, signal):
        """Performs pre-computations for per-segment approximation cost.

        NOTE: the number of dimensions of the signal and their ordering
        must match those of the nodes of the graph.
        The function eigh used below returns the eigenvector corresponding to 
        the ith eigenvalue in the ith column eigvect[:, i]

        Args:
            signal (array): of shape [n_samples, n_dim].
        """
        self.signal = signal
        # computation of the GFSS
        _, eigvects = eigh(self.graph_laplacian_mat)
        gft =  signal @ eigvects # equals signal.dot(eigvects) = eigvects.T.dot(signal.T).T
        self.gft_mean = np.mean(gft, axis=0)
        # computation of the per-segment cost utils
        self.gft_square_cumsum = np.concatenate([np.zeros((1, signal.shape[1])), np.cumsum((gft - self.gft_mean[None, :])**2, axis=0)], axis=0)
        return self

    def error(self, start, end):
        """

        Args:
            start (int): start of the segment
            end (int): end of the segment

        Returns:
            float: segment cost
        """
        if end - start < self.min_size:
            raise ValueError(f'end - start shoud be higher than {self.min_size}')
        sub_square_sum = self.gft_square_cumsum[end] - self.gft_square_cumsum[start]
        return (end  - start) * np.sum(np.log(sub_square_sum / (end - start)))


## Experimental setting v.0: synthetic data

### Observation on the minimum distance between consecutive change points (minimum segment length)

We require that the different change points $(t_1, \ldots, t_K)$ verify:

$$|t_{k+1} - t_k| >= l ~ \forall k \in [1, K-1] $$

where $l$ can be seen as the minimum admissible segment length. In this paragraph we give a meaningful lower bound of this parameter. Such lower bound is related to the computation of the cost functions over the segments $[a, b] \subset [0, T]$, namely the graph stationary normal cost function $c_s$ described above and the standard normal cost function $c_n$:

- $ c_n(y_{a}, \ldots, y_{b-1}) = (b - a) \log  \left[ \det \left( \hat \Sigma_{a..b} \right) \right]$
- $ c_s(y_{a}, \ldots, y_{b-1}) = (b - a) \sum_{n=1}^N \log \hat{\gamma}_{a.b}^{(n)} $

Based on the formula of the spectrogram $\hat{\gamma}_{a.b}$ given in the introduction, there is no numerical constraints for the feasibility of the computation. However, the $\log [ \det ( \cdot ) ]$ function used in the formula for $c_n$ should be applied to invertible matrices $\Sigma_{a..b}$ only. Therefore, we should focus on the conditions under which the matrix:

$$ \hat \Sigma_{a..b} = \frac{1}{b-a} \sum_{t=a}^{b-1} (y_t - \mu_T) (y_t - \mu_T)^T \quad \text{ with } y_t \sim \mathcal{N}(\mu, \Sigma)  $$

is invertible. Actually, such conditions have already been clearly stated in different works from Random Matrix Theory (RMT). For instance, it is shown in [[Izenman2008](#Izenman2008)] that $n \hat \Sigma_{a..b} \sim \mathcal{W}(b-a, \Sigma)$ follows the Wishart distribution. In this framework, it is possible to study the distribution of the eigenvalues of $\hat \Sigma_{a..b}$ and to deduce that: 

$$ \text{If } b-a > N \text{ with } N  \text{ the dimension of } y_t, \text{ then } \hat \Sigma_{a..b} \text{ is almost surely invertible }   $$

Conversely, it is possible to show that if $ b-a < N $ (the number of observations is lower then the number of variables), the matrix $\hat \Sigma_{a..b}$ is nalmost surely not invertible. This can be done by considering the family the first $(N+1)$ columns of $\hat \Sigma_{a..b}$.

Thus, the right lower-bound $l$ should be $\boxed{l = N}$, which is consistent with statement from [[Ryan2023](#Ryan2023)].

Note: with segments of length $l$, one is not guaranteed to compute good estimates of the covariance matrix, but at least such computations is almost surely admissible.

### Synthetic data generation

We design two graph generation scenarios.

1. Erdős–Rényi (ER) graphs with random parameters

In this scenario, we pick random number of nodes $N$ uniformly drawn in $[N_{\min}, N_{\max}]$, that may vary according to the experiment, and a random edge probability $p$ uniformly drawn in $[0.15, 0.5]$. The latter was chosen so that the generated graphs empirically have a realistic connectivity.

2. Geographic-like graphs 

We pick a random number $N$ of nodes uniformly drawn in $[N_{\min}, N_{\max}]$. The nodes are randomly located in $[0, 1]^2$ using the uniform law $\mathcal{U}([0, 1]^2)$. Eventually, we build the adjacency matrix of the graph by applying a threshold $\rho$ to the distance separating the nodes. More formally, if we denote $(W_{ij})_{1 \leq i, j \leq N}$ the coefficients of the adjacency matrix, we explore the following two formulas:

\begin{equation}
    W_{ij} = \mathbb{I} \left( \|p_i - p_j \|_2 \leq \rho \right)
\end{equation}

\begin{equation}
    W_{ij} = \frac{\rho}{\|p_i - p_j \|_2}  \mathbb{I} \left( \|p_i - p_j \|_2 \leq \rho \right)
\end{equation}

where $p_i$ denotes the 2D coordinates of node $i$. In the above formulas, the value of $\rho$ is chosen empirically so that the resulting graphs visually exhibit connectivity patterns that are consistent to what one may expect when using a graph structure for signal analysis. More precisely, we use the following values:

- for $N$ drawn in $[10, 50]$: $~$ $\rho = 0.3$
- for $N$ drawn in $[80, 110]$: $\rho = 0.2$

This scenario simulates 2D geographic graphs and is more likely to match realistic use-cases.


### Signal generation

Let $G$ be a graph randomly generated using one of the above scenarios. We recall that the laplacian matrix $L$ of the graph verifies $L = U \Lambda U^T$, where the columns of $U$ are the eigenvectors of $L$. We now describe how to generate a signal $(y_t)_{1 \leq t \leq T}$ that verifies the hypothesis presented in the [problem formulation](#problem-formulation).

We first pick an admissible number of change points $K$ depending on the signal length $T=1000$ and the minimum segment length $l = N$ by sampling $~  \mathcal{U}([1, \min(\frac{T}{10}, \frac{T}{l})])$. The change points $(t_k)_{1 \leq k \leq K}$ are uniformly drawn in $[l, T-l]$, by checking that a newly selected change point does not break the minimum segment length criteria. Therefore, it may happen that after a limit number of iterations, not $K$ change points were drawn, but we still use the same notations (SHOULD BE CHANGED).

Finally, we apply the following formula to generate the signal $(y_t)_{1 \leq t \leq T}$:



$$ \forall ~ k \in [1, K-1] ~ \forall  t \in [t_k, t_{k+1}] \quad  y_t \sim \mathcal{N}_N(0, \Sigma_k) \quad \text{ with } \quad \Sigma_k = U \text{diag}(\gamma_k) U^T ~  \text{ and } ~ \gamma_k \sim \mathcal{U}niform_N([0, 1]) $$

### Utils and visualization

#### Miscellaneous

In [ ]:
def get_git_head_short_hash() -> str:
    return subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD']).decode('ascii').strip()

In [ ]:
def turn_all_list_of_dict_into_str(data:dict):
    new_dict = {}
    for key, val in data.items():
        if isinstance(val, list):
            new_dict[key] = str(val)
        elif isinstance(val, dict):
            new_dict[key] = turn_all_list_of_dict_into_str(val)
        else:
            new_dict[key] = val
    return new_dict

In [ ]:
def turn_str_of_list_into_list_of_int(list_str):
    assert list_str[0] == '[' and list_str[-1] == ']'
    list_of_str = list_str[1:-1].split(',')
    return [int(val_str) for val_str in list_of_str]

def turn_str_of_list_into_list_of_float(list_str):
    assert list_str[0] == '[' and list_str[-1] == ']'
    list_of_str = list_str[1:-1].split(',')
    return [float(val_str) for val_str in list_of_str]

In [ ]:
def create_parent_and_dump_json(parent_dir, name, data, indent=None):
    if os.path.exists(os.path.join(parent_dir, name)):
        raise FileExistsError
    if not os.path.exists(parent_dir):
        Path(parent_dir).mkdir(parents=True, exist_ok=False)
    with open(os.path.join(parent_dir, name), 'w+') as f:
        json.dump(data, f, indent=indent)

In [ ]:
def open_json(path):
    with open(path, 'r+') as f:
        content = json.load(f)
    return content

#### Data generation

In [ ]:
def generate_random_er_graphs(params_rng, nx_graph_seed, max_n_nodes, min_n_nodes=10, min_edge_p=0.15, max_edge_p=0.5):
    n_nodes = params_rng.integers(low=min_n_nodes, high=max_n_nodes+1)
    edge_p = min_edge_p + (max_edge_p-min_edge_p) * params_rng.random()
    G = nx.erdos_renyi_graph(n=n_nodes, p=edge_p, seed=nx_graph_seed)
    return G

In [ ]:
graph_seed = 0
params_seed = 1
fig, axes = plt.subplots(1, 5, figsize=(4*5, 3))
params_rng = np.random.default_rng(seed=params_seed)
for _ in range(5):
    G = generate_random_er_graphs(params_rng, graph_seed, max_n_nodes=30)
    coord = nx.spring_layout(G, seed=0)
    nx.draw_networkx(G, with_labels=False, pos=coord, node_size=50, ax=axes[_])

In [ ]:
def generate_random_ba_graphs(params_rng, nx_graph_seed, min_n_nodes, max_n_nodes):
    n_nodes = params_rng.integers(low=min_n_nodes, high=max_n_nodes+1)
    m = int(params_rng.normal(loc=n_nodes//10, scale=1))
    G = nx.barabasi_albert_graph(n=n_nodes, m=m, seed=nx_graph_seed)
    return G

In [ ]:
graph_seed = 0
params_seed = 1
fig, axes = plt.subplots(1, 5, figsize=(4*5, 3))
params_rng = np.random.default_rng(seed=params_seed)
for _ in range(5):
    G = generate_random_ba_graphs(params_rng, graph_seed, min_n_nodes=10, max_n_nodes=30)
    coord = nx.spring_layout(G, seed=0)
    nx.draw_networkx(G, with_labels=False, pos=coord, node_size=50, ax=axes[_])

In [ ]:
def generate_random_weighted_geographic_graph(params_rng, min_n_nodes, max_n_nodes, dist_threshold):
    # generation of the nodes coordinates
    n_nodes = params_rng.integers(low=min_n_nodes, high=max_n_nodes+1)
    nodes_coord = params_rng.random(size=(n_nodes, 2))
    # computation of the adjacency matrix based on distance threshold 
    dist_mat_condensed = pdist(nodes_coord, metric='euclidean')
    adj_mat = squareform((dist_threshold/dist_mat_condensed)*(dist_mat_condensed < dist_threshold).astype(int))
    G = nx.Graph(adj_mat)
    return G, nodes_coord

def generate_random_geographic_graph(params_rng, min_n_nodes, max_n_nodes, dist_threshold):
    # generation of the nodes coordinates
    n_nodes = params_rng.integers(low=min_n_nodes, high=max_n_nodes+1)
    nodes_coord = params_rng.random(size=(n_nodes, 2))
    # computation of the adjacency matrix based on distance threshold 
    dist_mat_condensed = pdist(nodes_coord, metric='euclidean')
    adj_mat = squareform((dist_mat_condensed < dist_threshold).astype(int))
    G = nx.Graph(adj_mat)
    return G, nodes_coord

In [ ]:
graph_seed = 0
params_seed = 1
fig, axes = plt.subplots(1, 5, figsize=(5*4, 3))
params_rng = np.random.default_rng(seed=params_seed)
for _ in range(5):
    G, coord = generate_random_geographic_graph(params_rng, min_n_nodes=80, max_n_nodes=110, dist_threshold=0.20)
    coord_dic = {i: coord[i, :] for i in range(coord.shape[0])}
    nx.draw_networkx(G, with_labels=False, pos=coord_dic, node_size=50, ax=axes[_])

In [ ]:
graph_seed = 0
params_seed = 1
params_rng = np.random.default_rng(seed=params_seed)

n_nodes = params_rng.integers(low=80, high=110+1)
nodes_coord = params_rng.random(size=(n_nodes, 2))
# computation of the adjacency matrix based on distance threshold 
dist_mat_condensed = pdist(nodes_coord, metric='euclidean')
# --> to check that there is no 0 value in the condensed format

In [ ]:
def generate_gaus_signal_with_cov_diag_in_basis(n_dims, n_samples, basis, signal_rng, diag_cov_max=1):
    # randomly draw diagonal coef (in the fourier space)
    diag_coefs = diag_cov_max * signal_rng.random(n_dims)
    diag_mat = np.diag(diag_coefs)
    # compute the corresponding covariance matrix and signal 
    cov_mat = basis @ diag_mat @ basis.T
    signal = signal_rng.multivariate_normal(np.zeros(n_dims), cov_mat, size=n_samples)
    return signal

In [ ]:
seg_length = Literal["large", "minimal"]

def get_min_size_for_hyp(n_dims, hyp:seg_length):
    # the minimal segment length for admissible computations
    min_size = n_dims
    if hyp == "large":
        #for segment long enough for good estimates
        min_size = n_dims * (n_dims-1) / 2
    return min_size

In [ ]:
def draw_bkps_with_gap_constraint(n_samples, bkps_gap, bkps_rng, n_bkps_max, max_tries=10000):
    # randomly pick an admissible number of bkps
    n_bkps = bkps_rng.integers(low=1, high=min(n_bkps_max, n_samples // bkps_gap))
    bkps = []
    n_tries = 0
    # select admissible randomly drawn bkps
    while n_tries < max_tries and len(bkps) < n_bkps:
        new_bkp = bkps_rng.integers(low=bkps_gap, high=n_samples-bkps_gap)
        to_keep = True
        for bkp in bkps:
            if abs(new_bkp - bkp) < bkps_gap:
                to_keep = False
                break
        if to_keep:
            bkps.append(new_bkp)
        n_tries+=1
    bkps.sort()
    return bkps + [n_samples]

In [ ]:
def draw_fixed_nb_bkps_with_gap_constraint(n_samples, bkps_gap, bkps_rng, max_tries=10000):
    # randomly pick an admissible number of bkps
    n_bkps = bkps_rng.integers(low=n_samples // bkps_gap - 1, high= n_samples // bkps_gap)
    bkps = []
    n_tries = 0
    # select admissible randomly drawn bkps
    while n_tries < max_tries and len(bkps) < n_bkps:
        new_bkp = bkps_rng.integers(low=bkps_gap, high=n_samples-bkps_gap)
        to_keep = True
        for bkp in bkps:
            if abs(new_bkp - bkp) < bkps_gap:
                to_keep = False
                break
        if to_keep:
            bkps.append(new_bkp)
        n_tries+=1
    bkps.sort()
    return bkps + [n_samples]

#### Data writting and reading

In [ ]:
def save_data(G:nx.Graph, signal:np.ndarray, bkps:List[int], dir:str):
    # create subfolder
    Path(dir).mkdir(parents=True, exist_ok=False)
    # save graph
    adj_mat = nx.to_numpy_array(G)
    with open(f'{dir}/graph_adj_mat.npy', 'wb+') as nx_f:
        np.save(nx_f, adj_mat, allow_pickle=False)
    # save signal
    with open(f'{dir}/signal.npy', 'wb+') as np_f:
        np.save(np_f, signal, allow_pickle=False)
    # save bkps
    bkps_str = [int(bkp) for bkp in bkps]
    create_parent_and_dump_json(dir, "bkps.json", bkps_str)

In [ ]:
def read_data(path: str):
    adj_mat = np.load(f"{path}/graph_adj_mat.npy", allow_pickle=False)
    G = nx.from_numpy_array(adj_mat)
    signal = np.load(f"{path}/signal.npy", allow_pickle=False)
    bkps = open_json(f"{path}/bkps.json")
    return G, signal, bkps

#### Results computation and saving

In [ ]:
from ruptures.metrics import precision_recall
from ruptures.metrics import hausdorff

def compute_f1_score(preci, recall):
    if preci + recall > 0:
        return 2 * (preci * recall) / (preci + recall)
    return 0

def compute_and_update_metrics(true_bkps, pred_bkps, metrics_dic, prec_rec_margin):
    preci, recall = precision_recall(true_bkps, pred_bkps, prec_rec_margin)
    f1_score = compute_f1_score(preci, recall)
    hsdrf = hausdorff(true_bkps, pred_bkps)
    metrics_dic["precision"]['raw'].append(round(preci, 4))
    metrics_dic["recall"]['raw'].append(round(recall, 4))
    metrics_dic["f1_score"]['raw'].append(round(f1_score, 4))
    metrics_dic["hausdorff"]['raw'].append(int(hsdrf))

In [ ]:
def compute_and_add_stat_on_metrics(model_metrics:dict):
    for model_res in model_metrics.values():
        for metric_name, res in model_res.items():
            model_res[metric_name]['mean'] = round(np.mean(res['raw']), ndigits=4)
            model_res[metric_name]['std'] = round(np.std(res['raw']), ndigits=4)
    return model_metrics

In [ ]:
def save_metrics(metrics_per_models, stats, dir, comment=''):
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    to_save = {"date_time": now, 'comment': comment}
    to_save["hyper-parameters"] = stats
    to_save["results"] = metrics_per_models
    to_save = turn_all_list_of_dict_into_str(to_save)
    create_parent_and_dump_json(dir, now + '.json', to_save, indent=4)

#### Plotting utils

In [ ]:
def get_res_per_metric_per_cost_func(res_folder_list, cost_func_keys, metrics_keys, preci_margin):
    # initialization
    stat_per_metric_per_cost_func = {}
    for cost_func in cost_func_keys:
        stat_per_metric_per_cost_func[cost_func] = {}
        for metric in metrics_keys:
            stat_per_metric_per_cost_func[cost_func][metric] = {"mean": [], "std": [], "raw": []}
    # parsing metrics file to store results adequately
    for folder_name in res_folder_list:
        file_metric = open_json(os.path.join(folder_name, f"metrics_{preci_margin}.json"))
        for cost_func in cost_func_keys:
            for metric in metrics_keys:
                mean = file_metric[cost_func][metric]["mean"]
                stat_per_metric_per_cost_func[cost_func][metric]["mean"].append(mean)
                std = file_metric[cost_func][metric]["std"]
                stat_per_metric_per_cost_func[cost_func][metric]["std"].append(std)
                raw_str = file_metric[cost_func][metric]["raw"]
                raw = turn_str_of_list_into_list_of_float(raw_str)
                stat_per_metric_per_cost_func[cost_func][metric]["raw"].append(raw)
    return stat_per_metric_per_cost_func

In [ ]:
bar_width = 0.08
shift = 0.1
error_kw = {"capsize": 5, "elinewidth": 0.8, "capthick": 2}

def plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, met_name, ax, to_label):
    plot_x_coord = np.linspace(0, 1, num=(len(abscissa)+1))[:-1]
    for i, cost_func in enumerate(cost_func_keys):
        label = None
        if to_label:
            label = cost_func
        color = colors_per_cost_func[cost_func]
        # bar plot based on the statistics
        x = plot_x_coord + shift*i
        y = res_dic[cost_func][met_name]["mean"]
        y_err = res_dic[cost_func][met_name]["std"]
        ax.bar(x = x, height=y, yerr=y_err, label=label, width=bar_width, error_kw=error_kw, color=color)
        # scatter plot based on the raw values
        raw = np.array(res_dic[cost_func][met_name]["raw"])
        raw.flatten()
        x_scat_val = []
        for x_id in range(len(x)):
            x_scat_val = x_scat_val + [x[x_id]] * len(raw[x_id])
        ax.scatter(x=x_scat_val, y=raw, alpha=0.3, c='k', s=10)
    ax.set_xticks(plot_x_coord + shift*len(cost_func_keys)/2 - shift/2, abscissa)
    return ax

In [ ]:
def add_errorbar_manually(abscissa, y, yerr, marker, linestyle, linewidth, markersize, color, ax):
    # create top and bottom lim
    y_err_top = [y_pos + yerr_val for y_pos, yerr_val in zip(y, yerr)]
    y_err_bottom = [y_pos - yerr_val for y_pos, yerr_val in zip(y, yerr)]
    # add the top and bottom markers
    ax.scatter(abscissa, y_err_top, s=markersize, c=color, marker=marker)
    ax.scatter(abscissa, y_err_bottom, s=markersize, c=color, marker=marker)
    # add the vertical lines
    ax.vlines(x=abscissa, ymin=y_err_bottom, ymax=y_err_top, color=color, linewidth=linewidth, linestyle=linestyle)
    return ax

In [ ]:
def plot_scatter_errorbar(abscissa_pos, cost_func_keys, color, markers, linestyles, met_name, res_dic, ax):
    for i, cost_func in enumerate(cost_func_keys):
        y = res_dic[cost_func][met_name]["mean"]
        y_err = res_dic[cost_func][met_name]["std"]
        ax.plot(abscissa_pos, y, c=color, marker=markers[i], markersize=7, linewidth=2, linestyle=linestyles[i])
        add_errorbar_manually(abscissa_pos, y, y_err, marker=markers[i], linestyle=linestyles[i], linewidth=1, markersize=20, color=color, ax=ax)
    return ax

In [ ]:
##### INITIALIZATION
preci_margin = 5
res_folder_list = [
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop002",
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop005",
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop01",
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop02",
]

cost_func_keys = ["statio normal cost", "normal cost"]
markers = ['o', "s"]
linestyles = ["dashed", "dotted"]
metrics_keys = ["f1_score", "hausdorff"]
abscissa_pos =  [2, 4, 6, 8]
graph_types = ["exp geo", "KNN geo"]
graph_colors = ['darkorange', 'dodgerblue']
shift = 0.2

res_dic = get_res_per_metric_per_cost_func(res_folder_list, cost_func_keys, metrics_keys, preci_margin)

##### PLOTTING
fig, axes = plt.subplots(1, (len(metrics_keys)), figsize=(7*len(metrics_keys), 5)) #, layout='constrained')

for i in range(len(graph_types)):
    x_coords = [x + i*shift for x in abscissa_pos]
    draw_ticks = i==0
    plot_scatter_errorbar(x_coords, cost_func_keys, graph_colors[i], markers, linestyles, "f1_score", res_dic, ax=axes[0])
    axes[0].set_xlabel("Number of nodes")
    axes[0].set_title('F1 SCORE per cost function')
    plot_scatter_errorbar(x_coords, cost_func_keys, graph_colors[i], markers, linestyles, "hausdorff", res_dic, ax=axes[1])
    axes[1].set_xlabel("Number of nodes")
    axes[1].set_title('Haussdorff per cost function')
    axes[1].set_ylim(bottom=0)

axes[0].set_xticks(abscissa_pos, abscissa_pos)
axes[1].set_xticks(abscissa_pos, abscissa_pos)

legend_elements = [
    Line2D([0], [0], color='k', lw=0.01, marker=markers[0], linestyle=linestyles[0], label=cost_func_keys[0]),
    Line2D([0], [0], color='k', lw=0.01, marker=markers[1], linestyle=linestyles[1], label=cost_func_keys[1]),
    Line2D([0], [0], color='darkorange', lw=3, label="Exp sim geo graphs"),
]               
fig.suptitle("Comparison")
fig.subplots_adjust(bottom=0.17)
fig.legend(handles=legend_elements, loc="lower center", ncols=len(legend_elements))

In [ ]:
##### INITIALIZATION
res_folder_list = [
    "results/synthetic/A_within_hypothesis/A_5/geo_min_gap_weighted",
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop01",
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop02"
]
abscissa =  [0.05, 0.1, 0.2]
cost_func_keys = ["statio normal cost", "normal cost"]
colors_per_cost_func = {"statio normal cost": "darkorange", "normal cost": "dodgerblue"}
metrics_keys = ["precision", "hausdorff"]
res_dic = get_res_per_metric_per_cost_func(res_folder_list, cost_func_keys, metrics_keys, preci_margin=5)

##### PLOTTING
fig, axes = plt.subplots(1, (len(metrics_keys)), figsize=(5*len(metrics_keys), 6)) #, layout='constrained')
# precision
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "precision", axes[0], to_label=True)
axes[0].set_xlabel("proportion")
axes[0].set_title('Precision per cost function')
# haussdorff
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "hausdorff", axes[1],  to_label=False)
axes[1].set_xlabel("proportion")
axes[1].set_title('Haussdorff per cost function')
axes[1].set_ylim(bottom=0)

fig.suptitle("Comparison")
fig.legend(loc=(0.3, 0), ncols=len(cost_func_keys))


### A. Data verifying the hypothesis of the model

In this experiment, we generate data according to the hypothesis presented in the [problem formulation](#problem-formulation) and we compare our method to the cost function for standard covariance change detection in Gaussian models (that is supposed to cover our hypothesis). More precisely, we randomly generate a graph and a corresponding multivariate Gaussian signal, undergoing a (known) random number of covariance change points. The comparison between the two methods relies on the precision / recall and Hausdorff metrics.

In [ ]:
def generate_rd_signal_in_hyp_with_max_tries(G:nx.Graph, signal_rng:np.random.Generator, n_bkps_max, hyp:seg_length, n_samples:int, diag_cov_max=1):
    # randomly draw a set of admissible change points
    n_dims = G.number_of_nodes()
    min_size = get_min_size_for_hyp(n_dims=n_dims, hyp=hyp)
    bkps = draw_bkps_with_gap_constraint(n_samples, min_size, signal_rng, n_bkps_max)
    # generate the signal
    _, eigvects = eigh(nx.laplacian_matrix(G).toarray())
    signal_gen_func = lambda size: generate_gaus_signal_with_cov_diag_in_basis(n_dims, size, eigvects, signal_rng, diag_cov_max)
    signal = signal_gen_func(bkps[0])
    # add each sub-segment
    for i in range(1, len(bkps)):
        sub_signal = signal_gen_func(bkps[i] - bkps[i-1])
        signal = np.concatenate([signal, sub_signal], axis=0)
    return bkps, signal.astype(np.float64)

In [ ]:
nx_graph_seed = 1
graph_seed = 2
signal_seed = 3

signal_rng = np.random.default_rng(seed=signal_seed)
graph_rng = np.random.default_rng(seed=graph_seed)

# G = generate_random_er_graphs(graph_rng, nx_graph_seed, max_n_nodes=30)
G, _ = generate_random_weighted_geographic_graph(graph_rng, min_n_nodes=40, max_n_nodes=50, dist_threshold=0.30)
bkps, s = generate_rd_signal_in_hyp_with_max_tries(G, signal_rng, n_samples=1000, diag_cov_max=10, hyp='min', n_bkps_max=10)

fig, axes = plt.subplot_mosaic(mosaic='ABB', figsize=(14, 3))
nx.draw_networkx(G, ax=axes['A'])
for i in range(6):
    axes['B'].plot(10*i+s[:, i])
axes['B'].set_yticks([])
for bkp in bkps[:-1]:
    axes['B'].axvline(x=bkp, c='k')

### B. Data not verifying the hypothesis of the model at all

In what follows, we work with signals verifying hypothesis 1 from the [the problem formulation](#problem-formulation), but not respecting the second hypothesis. More precisely, we will generate covariance matrices that are diagonalizable in a basis different from the Fourier basis $U$. To do so, we generate a graph $G'$  different from the graph $G$ used to compute the cost function. Then, we apply the same process and formula as previously described to generate the signal but using the eigenvectors $U'$ of the laplacian matrix $L'$ of $G'$.

In [ ]:
def generate_rd_signal_from_other_basis(G:nx.Graph, signal_rng:np.random.Generator, hyp:seg_length, n_bkps_max, n_samples, diag_cov_max=1):
    # randomly draw a set of admissible change points
    n_dims = G.number_of_nodes()
    min_size = get_min_size_for_hyp(n_dims=n_dims, hyp=hyp)
    bkps = draw_bkps_with_gap_constraint(n_samples, min_size, signal_rng, n_bkps_max)
    # generate another graph to compute the signal covariance matrices
    G_for_cov = generate_random_er_graphs(signal_rng, NX_GRAPH_SEED, max_n_nodes=n_dims, min_n_nodes=n_dims, min_edge_p=0.01, max_edge_p=1)
    _, eigvects = eigh(nx.laplacian_matrix(G_for_cov).toarray())
    signal_gen_func = lambda size: generate_gaus_signal_with_cov_diag_in_basis(n_dims, size, eigvects, signal_rng, diag_cov_max)
    signal = signal_gen_func(bkps[0])
    # add each sub-segment
    for i in range(1, len(bkps)):
        sub_signal = signal_gen_func(bkps[i] - bkps[i-1])
        signal = np.concatenate([signal, sub_signal], axis=0)
    return bkps, signal

In [ ]:
N_EXP = 100
NX_GRAPH_SEED = 1
GRAPH_SEED = 2
SIGNAL_SEED = 3
MIN_N_NODES = 80
MAX_N_NODES = 110
DIST_THRESHOLD = 0.2
N_SAMPLES = 1000
DIAG_COV_MAX = 1
MAX_N_BKPS = 10
BKPS_GAP_CONSTRAINT: seg_length = "minimal" 

###################################################################
NAME = "weighted_geo_min_gap_cons"
data_dir = "data/synthetic_data/exp_B_totally_out_of_hypothesis/B_3/" + NAME
desc = ['Minimal gap constraint, using weighted geographical graphs', 'The covariance matrix of the signal is diagonal in the Fourier basis of a random Erdos-Renyi graphs', 'Purpose: check performance when totally out of the model.']
## FUNCTIONS USED FOR GRAPH GENERATION
###################################################################

# initialization
signal_rng = np.random.default_rng(seed=SIGNAL_SEED)
graph_rng = np.random.default_rng(seed=GRAPH_SEED)
data_metadata = {"description": desc, "commit hash":get_git_head_short_hash(), "graph_func": generate_random_weighted_geographic_graph.__name__, "signal_func": generate_rd_signal_from_other_basis.__name__, "n_iter": N_EXP, "graph_seed": GRAPH_SEED, "nx_graph_seed": NX_GRAPH_SEED, "signal_seed": SIGNAL_SEED, "n_samples": N_SAMPLES, "min_n_nodes": MIN_N_NODES, "max_n_nodes": MAX_N_NODES, "dist_threshold": DIST_THRESHOLD, "max_n_bkps": MAX_N_BKPS, "bkps_gap_hyp": BKPS_GAP_CONSTRAINT, "diag_cov_max": DIAG_COV_MAX, "n_nodes": [], 'n_bkps': []}

# data generation and saving
for exp_id in range(N_EXP):
    subdir = f"{data_dir}/{exp_id}"
    G, _ = generate_random_weighted_geographic_graph(graph_rng, min_n_nodes=MIN_N_NODES, max_n_nodes=MAX_N_NODES, dist_threshold=DIST_THRESHOLD)
    data_metadata["n_nodes"].append(int(G.number_of_nodes()))
    gt_bkps, signal = generate_rd_signal_from_other_basis(G, signal_rng, hyp=BKPS_GAP_CONSTRAINT, n_bkps_max=MAX_N_BKPS, n_samples=N_SAMPLES, diag_cov_max=DIAG_COV_MAX)
    data_metadata["n_bkps"].append(len(gt_bkps)-1)
    save_data(G, signal, gt_bkps, subdir)

# storing metadata
data_metadata = turn_all_list_of_dict_into_str(data_metadata)
create_parent_and_dump_json(data_dir, "metadata.json", data_metadata, indent=4)

In [ ]:
# initialization
DATE = None
if DATE :
    date = "2024-06-18_17-22-19"
    data_dir = "data/synthetic_data/exp_B_totally_out_of_hypothesis/B_1/" + date

################################################################
results_dir = "results/synthetic/B_totally_out_of_hypothesis/B_3/" + NAME
data_dir = data_dir
################################################################

data_stats = open_json(f"{data_dir}/metadata.json")
data_stats["bkps"] = {}
data_stats["data_path"] = data_dir

# output formatting
statio_results = {}
normal_results = {}

subfold_list = [int(subdir) for subdir in os.listdir(data_dir) if '.' not in subdir]
subfold_list.sort()
for exp_id in tqdm(subfold_list, desc='Running experiment...'):
    
    exp_id = str(exp_id) # just for compatibility

    # loading data
    subdir = f"{data_dir}/{exp_id}"
    G, signal, gt_bkps = read_data(subdir)
    min_size = get_min_size_for_hyp(G.number_of_nodes(), hyp=data_stats["bkps_gap_hyp"])
    data_stats["bkps"][exp_id] = gt_bkps

    # prediction with standard normal cost
    t1 = time.perf_counter()
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        normal_cost = rpt.costs.CostNormal()
        algo_normal = rpt.Dynp(custom_cost=normal_cost, jump=1, min_size=min_size).fit(signal)
        normal_bkps = algo_normal.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    normal_results[exp_id] = {}
    normal_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    normal_results[exp_id]["pred"] = normal_bkps

    # prediction with stationary normal cost
    t1 = time.perf_counter()
    statio_cost = CostGraphStatioNormal(nx.laplacian_matrix(G).toarray())
    algo_statio = rpt.Dynp(custom_cost=statio_cost, jump=1, min_size=min_size).fit(signal)
    statio_bkps = algo_statio.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    statio_results[exp_id] = {}
    statio_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    statio_results[exp_id]["pred"] = statio_bkps

# storing
create_parent_and_dump_json(results_dir, "data_stats.json", turn_all_list_of_dict_into_str(data_stats), indent=4)
create_parent_and_dump_json(results_dir, "statio_pred.json", turn_all_list_of_dict_into_str(statio_results), indent=4)
create_parent_and_dump_json(results_dir, "normal_pred.json", turn_all_list_of_dict_into_str(normal_results), indent=4)


In [ ]:
PRECI_RECALL_MARGIN = 2

#############################################################
pred_dir = results_dir
#############################################################

# initialization
data_stats = open_json(f"{results_dir}/data_stats.json")
statio_pred_dic = open_json(f"{results_dir}/statio_pred.json")
normal_pred_dic = open_json(f"{results_dir}/normal_pred.json")
gt_bkps_dic = data_stats.pop('bkps')

# output formatting
metrics_dic = {}
metrics_dic["pred_path"] = results_dir
metrics_dic["hyper-parameters"] = data_stats
metrics_dic["hyper-parameters"]["metrics_margin"] = PRECI_RECALL_MARGIN

statio_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}
normal_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}

for exp_id in gt_bkps_dic.keys():
    # compute metrics
    normal_pred_bkps = turn_str_of_list_into_list_of_int(normal_pred_dic[exp_id]["pred"])
    statio_pred_bkps = turn_str_of_list_into_list_of_int(statio_pred_dic[exp_id]["pred"])
    gt_bkps = turn_str_of_list_into_list_of_int(gt_bkps_dic[exp_id])
    compute_and_update_metrics(gt_bkps, normal_pred_bkps, normal_results, PRECI_RECALL_MARGIN)
    compute_and_update_metrics(gt_bkps, statio_pred_bkps, statio_results, PRECI_RECALL_MARGIN)
    # add time values
    normal_results["time"]["raw"].append(normal_pred_dic[exp_id]["time"])
    statio_results["time"]["raw"].append(statio_pred_dic[exp_id]["time"])

# results post-precessing and saving
full_results = {"statio normal cost": statio_results, "normal cost": normal_results}
full_results = compute_and_add_stat_on_metrics(full_results)
full_results = turn_all_list_of_dict_into_str(full_results)
create_parent_and_dump_json(pred_dir, f'metrics_{PRECI_RECALL_MARGIN}.json', full_results, indent=4)

In [ ]:
##### INITIALIZATION
PRECI_RECALL_MARGIN = 2
res_folder_list = [
    "results/synthetic/B_totally_out_of_hypothesis/B_4/non_weighted_geo_min_gap_cons",
    "results/synthetic/B_totally_out_of_hypothesis/B_3/geo_min_gap_cons_weighted",
    "results/synthetic/B_totally_out_of_hypothesis/B_5/weighted_geo_min_gap_cons",
]
abscissa =  ["big non weighted", "big weighted", "medium weighted"]
cost_func_keys = ["statio normal cost", "normal cost"]
colors_per_cost_func = {"statio normal cost": "darkorange", "normal cost": "dodgerblue"}
metrics_keys = ["precision", "hausdorff"]
res_dic = get_res_per_metric_per_cost_func(res_folder_list, cost_func_keys, metrics_keys, preci_margin=PRECI_RECALL_MARGIN)

##### PLOTTING
fig, axes = plt.subplots(1, (len(metrics_keys)), figsize=(6*len(metrics_keys), 5)) #layout='constrained')
# precision
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "precision", axes[0], to_label=True)
axes[0].set_title('Precision per cost function')
axes[0].set_ylim(bottom=0)
# haussdorff
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "hausdorff", axes[1],  to_label=False)
axes[1].set_title('Haussdorff per cost function')
axes[1].set_ylim(bottom=0)
axes[1].set_ylim(top=300)

N_min = 80
N_max = 110
graph_type = "(non-) / weighted geographic"
gap_hypothesis = "minimal"
N_iter = 100
T = 1000

# fig.suptitle(f"T={T}, N_min={N_min}, N_max={N_max}, {graph_type} graphs, {N_iter} iterations, margin={PRECI_RECALL_MARGIN}")
fig.suptitle(f"T={T}, [N_min, N_max] varies, {graph_type} graphs, {N_iter} iterations, margin={PRECI_RECALL_MARGIN}")
fig.legend(loc=(0.3, 0), ncols=len(cost_func_keys))

save_path = "images/results/synthetic/B_out_of_hyp/graph_type"
if not os.path.exists(save_path):
    Path(save_path).mkdir(parents=True, exist_ok=False)
name = 'big_medium_geo.png'
fig.savefig(os.path.join(save_path, name))

### C. Data verifying the hypothesis of the models, with node dropping to simulate breakdowns

In what follows, we work with signals verifying the two hypothesis from the [the problem formulation](#problem-formulation). Additionally, we will randomly select a (very) small number of nodes and simulate the breakdown of the corresponding sensors by setting the value of the signal lying on this node to 0 for a random time length. 

More formally, let denote $\eta_{max}$ the hyper-parameter corresponding to the maximal proportion of nodes that undergo a breakdown. We denote $N^{max}_{broken} = \lfloor \eta_{max} * N \rfloor$. For each signal $(y_t)_{1 \leq t \leq T} \in \mathbb{R}^{T \times N}$ , we draw $N_{broken}$ number of nodes undergoing a breakdown in $ ~  \mathcal{U}niform([0, N^{max}_{broken}])$. Then, for a node $i$ undergoing a breakdown we apply:

\begin{equation}
    t_{start} ~ \sim ~ \mathcal{U}niform([0, T-1]), ~ t_{end} ~ \sim ~ \mathcal{U}niform([t_{start}, T]) \qquad \forall ~  t \in [t_{start}, t_{end}] ~ y_t^i = 0
\end{equation}

Therefore, by increasing the value of $\eta_{max}$ we evaluate the robustness of our cost function with respect to brutal, isolated and uncorrelated mean changes.

In [ ]:
def modify_signal_to_simulate_breakdown(signal, signal_rng, n_breakdown_max):
    # initialization
    n_samples = signal.shape[0]
    n_breakdown = signal_rng.integers(1, n_breakdown_max+1)
    # randomly pick the location and time length of the breakdowns
    breakdowns = {}
    broken_node_ids = signal_rng.integers(0, signal.shape[1], size=(n_breakdown))
    for node_id in broken_node_ids:
        start = int(signal_rng.integers(0, n_samples-1))
        end = int(signal_rng.integers(start, n_samples))
        signal[start:end, node_id] = 0
        breakdowns[int(node_id)] = (start, end)
    return signal, breakdowns

In [ ]:
nx_graph_seed = 1
graph_seed = 2
signal_seed = 5

signal_rng = np.random.default_rng(seed=signal_seed)
graph_rng = np.random.default_rng(seed=graph_seed)

G = generate_random_er_graphs(graph_rng, nx_graph_seed, max_n_nodes=30)
bkps, s = generate_rd_signal_in_hyp_with_max_tries(G, signal_rng, n_bkps_max=10, n_samples=G.number_of_nodes()**2, hyp='minimal', diag_cov_max=1)
s, breakdowns = modify_signal_to_simulate_breakdown(s, signal_rng, n_breakdown_max=G.number_of_nodes()//10)

print("The generated breakdowns are:", breakdowns)

fig, ax = plt.subplots(1, 1, figsize=(12,3))
for i in range(5):
    ax.plot(10*i+s[:, i])
for i, node_id in enumerate(breakdowns.keys()):
    ax.plot(10*(i+5)+s[:, node_id])

In [ ]:
N_EXP = 100
NX_GRAPH_SEED = 1
GRAPH_SEED = 2
SIGNAL_SEED = 3
MIN_N_NODES = 40
MAX_N_NODES = 50
DIST_THRESHOLD = 0.3
N_SAMPLES = 1000
PROP_N_NODES_BREAKDOWN = 0.4
DIAG_COV_MAX = 1
MAX_N_BKPS = 10
BKPS_GAP_CONSTRAINT: seg_length = "minimal" 

###################################################################
NAME = "min_gap_cons_non_weighted_geo_prop04"
data_dir = "data/synthetic_data/exp_C_censor_breakdown_in_hypothesis/C_5/" + NAME
desc = ['Minimal gap constraint, using geographical non-weighted graphs, with high number of nodes', 'The signal is generated withing the model hypothesis but we simulate random rare censor breakdowns', 'Purpose: check if the cost functions are robust w.r.t isolated and rare censor breakdowns.']
## FUNCTIONS USED FOR GRAPH GENERATION
###################################################################

# initialization
signal_rng = np.random.default_rng(seed=SIGNAL_SEED)
graph_rng = np.random.default_rng(seed=GRAPH_SEED)
data_metadata = {"description": desc, "commit hash":get_git_head_short_hash(), "graph_func": generate_random_geographic_graph.__name__, "signal_func": generate_rd_signal_in_hyp_with_max_tries.__name__ + " + " + modify_signal_to_simulate_breakdown.__name__, "n_iter": N_EXP, "graph_seed": GRAPH_SEED, "nx_graph_seed": NX_GRAPH_SEED, "signal_seed": SIGNAL_SEED, "n_samples": N_SAMPLES, "min_n_nodes": MIN_N_NODES, "max_n_nodes": MAX_N_NODES, "dist_threshold": DIST_THRESHOLD, "max_n_bkps": MAX_N_BKPS, "bkps_gap_hyp": BKPS_GAP_CONSTRAINT, "diag_cov_max": DIAG_COV_MAX, "prop_n_nodes_breakdown": PROP_N_NODES_BREAKDOWN, "n_nodes": [], 'n_bkps': [], "breakdowns": []}

# data generation and saving
for exp_id in range(N_EXP):
    subdir = f"{data_dir}/{exp_id}"
    G, _ = generate_random_geographic_graph(graph_rng, min_n_nodes=MIN_N_NODES, max_n_nodes=MAX_N_NODES, dist_threshold=DIST_THRESHOLD)
    n_nodes = int(G.number_of_nodes())
    data_metadata["n_nodes"].append(n_nodes)
    gt_bkps, signal = generate_rd_signal_in_hyp_with_max_tries(G, signal_rng, n_bkps_max=MAX_N_BKPS, hyp=BKPS_GAP_CONSTRAINT, n_samples=N_SAMPLES, diag_cov_max=DIAG_COV_MAX)
    signal, breakdowns = modify_signal_to_simulate_breakdown(signal, signal_rng, int(n_nodes*PROP_N_NODES_BREAKDOWN))
    data_metadata["n_bkps"].append(len(gt_bkps)-1)
    data_metadata["breakdowns"].append(breakdowns)
    save_data(G, signal, gt_bkps, subdir)

# storing metadata
data_metadata = turn_all_list_of_dict_into_str(data_metadata)
create_parent_and_dump_json(data_dir, "metadata.json", data_metadata, indent=4)

In [ ]:
# initialization
DATE = None
if DATE :
    date = "2024-06-18_17-22-19"
    data_dir = "data/synthetic_data/exp_B_totally_out_of_hypothesis/B_1/" + date

################################################################
results_dir = "results/synthetic/C_censor_breakdown_in_hypothesis/C_5/" + NAME
data_dir = data_dir
################################################################

data_stats = open_json(f"{data_dir}/metadata.json")
data_stats["bkps"] = {}
data_stats["data_path"] = data_dir

# output formatting
statio_results = {}
normal_results = {}

subfold_list = [int(subdir) for subdir in os.listdir(data_dir) if '.' not in subdir]
subfold_list.sort()
for exp_id in tqdm(subfold_list, desc='Running experiment...'):

    exp_id = str(exp_id) # just for compatibility

    # loading data
    subdir = f"{data_dir}/{exp_id}"
    G, signal, gt_bkps = read_data(subdir)
    min_size = get_min_size_for_hyp(G.number_of_nodes(), hyp=data_stats["bkps_gap_hyp"])
    data_stats["bkps"][exp_id] = gt_bkps

    # prediction with standard normal cost
    t1 = time.perf_counter()
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        normal_cost = rpt.costs.CostNormal()
        algo_normal = rpt.Dynp(custom_cost=normal_cost, jump=1, min_size=min_size).fit(signal)
        normal_bkps = algo_normal.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    normal_results[exp_id] = {}
    normal_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    normal_results[exp_id]["pred"] = normal_bkps

    # prediction with stationary normal cost
    t1 = time.perf_counter()
    statio_cost = CostGraphStatioNormal(nx.laplacian_matrix(G).toarray())
    algo_statio = rpt.Dynp(custom_cost=statio_cost, jump=1, min_size=min_size).fit(signal)
    statio_bkps = algo_statio.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    statio_results[exp_id] = {}
    statio_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    statio_results[exp_id]["pred"] = statio_bkps

# storing
create_parent_and_dump_json(results_dir, "data_stats.json", turn_all_list_of_dict_into_str(data_stats), indent=4)
create_parent_and_dump_json(results_dir, "statio_pred.json", turn_all_list_of_dict_into_str(statio_results), indent=4)
create_parent_and_dump_json(results_dir, "normal_pred.json", turn_all_list_of_dict_into_str(normal_results), indent=4)


In [ ]:
PRECI_RECALL_MARGIN = 2

#############################################################
pred_dir = results_dir
#############################################################

# initialization
data_stats = open_json(f"{results_dir}/data_stats.json")
statio_pred_dic = open_json(f"{results_dir}/statio_pred.json")
normal_pred_dic = open_json(f"{results_dir}/normal_pred.json")
gt_bkps_dic = data_stats.pop('bkps')

# output formatting
metrics_dic = {}
metrics_dic["pred_path"] = results_dir
metrics_dic["hyper-parameters"] = data_stats
metrics_dic["hyper-parameters"]["metrics_margin"] = PRECI_RECALL_MARGIN

statio_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}
normal_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}

for exp_id in gt_bkps_dic.keys():
    # compute metrics
    normal_pred_bkps = turn_str_of_list_into_list_of_int(normal_pred_dic[exp_id]["pred"])
    statio_pred_bkps = turn_str_of_list_into_list_of_int(statio_pred_dic[exp_id]["pred"])
    gt_bkps = turn_str_of_list_into_list_of_int(gt_bkps_dic[exp_id])
    compute_and_update_metrics(gt_bkps, normal_pred_bkps, normal_results, PRECI_RECALL_MARGIN)
    compute_and_update_metrics(gt_bkps, statio_pred_bkps, statio_results, PRECI_RECALL_MARGIN)
    # add time values
    normal_results["time"]["raw"].append(normal_pred_dic[exp_id]["time"])
    statio_results["time"]["raw"].append(statio_pred_dic[exp_id]["time"])

# results post-precessing and saving
full_results = {"statio normal cost": statio_results, "normal cost": normal_results}
full_results = compute_and_add_stat_on_metrics(full_results)
full_results = turn_all_list_of_dict_into_str(full_results)
create_parent_and_dump_json(pred_dir, f'metrics_{PRECI_RECALL_MARGIN}.json', full_results, indent=4)

In [ ]:
##### INITIALIZATION
PRECI_RECALL_MARGIN = 2
res_folder_list = [
    "results/synthetic/C_censor_breakdown_in_hypothesis/C_3/min_gap_cons_weighted_geo_prop002",
    "results/synthetic/C_censor_breakdown_in_hypothesis/C_3/min_gap_cons_weighted_geo_prop01",
    "results/synthetic/C_censor_breakdown_in_hypothesis/C_3/min_gap_cons_weighted_geo_prop02",
    "results/synthetic/C_censor_breakdown_in_hypothesis/C_3/min_gap_cons_weighted_geo_prop05",
]
abscissa =  ["eta = 0.02", "eta = 0.1", "eta = 0.2", "eta = 0.5"]
cost_func_keys = ["statio normal cost", "normal cost"]
colors_per_cost_func = {"statio normal cost": "darkorange", "normal cost": "dodgerblue"}
metrics_keys = ["precision", "hausdorff"]
res_dic = get_res_per_metric_per_cost_func(res_folder_list, cost_func_keys, metrics_keys, preci_margin=PRECI_RECALL_MARGIN)

##### PLOTTING
fig, axes = plt.subplots(1, (len(metrics_keys)), figsize=(6*len(metrics_keys), 5)) #layout='constrained')
# precision
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "precision", axes[0], to_label=True)
axes[0].set_title('Precision per cost function')
axes[0].set_ylim(bottom=0)
# haussdorff
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "hausdorff", axes[1],  to_label=False)
axes[1].set_title('Haussdorff per cost function')
axes[1].set_ylim(bottom=0)
axes[1].set_ylim(top=300)

N_min = 80
N_max = 110
graph_type = "weighted geographic"
gap_hypothesis = "minimal"
N_iter = 100
T = 1000

fig.suptitle(f"T={T}, N_min={N_min}, N_max={N_max}, {graph_type} graphs, {N_iter} iterations, margin={PRECI_RECALL_MARGIN}")
fig.legend(loc=(0.3, 0), ncols=len(cost_func_keys))

save_path = "images/results/synthetic/C_censor_breakdown/nodes_prop"
if not os.path.exists(save_path):
    Path(save_path).mkdir(parents=True, exist_ok=False)
name = 'geo_weighted.png'
fig.savefig(os.path.join(save_path, name))

### D. Robustness with respect to (spatially and temporaly) independent additive white noise

In this experiment, we keep using signals that verify our two hypothesis. Though we add a temporally independent white noise with scalar covariance matrix to such signal. More formally, we apply the change point detection algorithm to the signal $(y'_t)_{1 \leq t \leq T}$ defined by

\begin{equation}
    \forall ~  t \in [0, T] \quad y'_t = y_t + e_t \quad \text{ with } \quad e_t \sim \mathcal{N}_N(0, \sigma)
\end{equation}

We evaluate the performance of our cost function against increasing value of $\sigma$.

In [ ]:
def add_diagonal_white_noise(signal_rng:np.random.Generator, signal, sigma):
    n_samples , n_dims = signal.shape
    cov_mat = sigma * np.eye(n_dims)
    white_noise = signal_rng.multivariate_normal(np.zeros(n_dims), cov_mat, size=n_samples)
    return signal + white_noise

In [ ]:
nx_graph_seed = 1
graph_seed = 2
signal_seed = 5

signal_rng = np.random.default_rng(seed=signal_seed)
graph_rng = np.random.default_rng(seed=graph_seed)

G, _ = generate_random_weighted_geographic_graph(graph_rng, min_n_nodes=10, max_n_nodes=30, dist_threshold=0.3)
bkps, s = generate_rd_signal_in_hyp_with_max_tries(G, signal_rng, n_bkps_max=10, n_samples=G.number_of_nodes()**2, hyp='minimal', diag_cov_max=1)
s_noise = add_diagonal_white_noise(signal_rng, s, sigma=3)

fig, axes = plt.subplots(1, 2, figsize=(12,2))
for i in range(5):
    axes[0].plot(10*i+s[:, i])
for i in range(5):
    axes[1].plot(10*i+s_noise[:, i])

In [ ]:
N_EXP = 100
NX_GRAPH_SEED = 1
GRAPH_SEED = 2
SIGNAL_SEED = 3
MIN_N_NODES = 40
MAX_N_NODES = 50
DIST_THRESHOLD = 0.3
DIAG_COV_MAX = 1
N_SAMPLES = 1000
MAX_N_BKPS = 10
SIGNAL_TO_NOISE_RATIO = 4
BKPS_GAP_CONSTRAINT: seg_length = "minimal" 

###################################################################
NAME = "min_gap_cons_non_weighted_geo_snr4"
data_dir = "data/synthetic_data/exp_D_in_hypo_diag_white_noise/D_5/" + NAME
desc = ['Minimal gap constraint, using non-weighted geographical graphs.', 'The signal is generated withing the model hypothesis but we add temporally and spatially independent white noise, meaning that the white noise follows multivariate gaussian distribution with scalar covariance matrix', 'Purpose: check if the cost functions are sensitive to additive white noise with scalar matrix covariance (none of the nodes are linked).']
## FUNCTIONS USED FOR GRAPH GENERATION
###################################################################

# initialization
signal_rng = np.random.default_rng(seed=SIGNAL_SEED)
graph_rng = np.random.default_rng(seed=GRAPH_SEED)
data_metadata = {"description": desc, "commit hash":get_git_head_short_hash(), "graph_func": generate_random_geographic_graph.__name__, "signal_func": generate_rd_signal_in_hyp_with_max_tries.__name__ + " + " + add_diagonal_white_noise.__name__, "n_iter": N_EXP, "graph_seed": GRAPH_SEED, "nx_graph_seed": NX_GRAPH_SEED, "signal_seed": SIGNAL_SEED, "n_samples": N_SAMPLES, "min_n_nodes": MIN_N_NODES, "dist_threshold": DIST_THRESHOLD, "max_n_nodes": MAX_N_NODES, "max_n_bkps": MAX_N_BKPS, "bkps_gap_hyp": BKPS_GAP_CONSTRAINT, "diag_cov_max": DIAG_COV_MAX, "signal_to_noise_ratio": SIGNAL_TO_NOISE_RATIO, "n_nodes": [], 'n_bkps': []}

# data generation and saving
for exp_id in range(N_EXP):
    subdir = f"{data_dir}/{exp_id}"
    G, _ = generate_random_geographic_graph(graph_rng, min_n_nodes=MIN_N_NODES, max_n_nodes=MAX_N_NODES, dist_threshold=DIST_THRESHOLD)
    n_nodes = int(G.number_of_nodes())
    data_metadata["n_nodes"].append(n_nodes)
    gt_bkps, signal = generate_rd_signal_in_hyp_with_max_tries(G, signal_rng, n_bkps_max=MAX_N_BKPS, hyp=BKPS_GAP_CONSTRAINT, n_samples=N_SAMPLES, diag_cov_max=DIAG_COV_MAX)
    signal = add_diagonal_white_noise(signal_rng, signal, sigma=DIAG_COV_MAX/SIGNAL_TO_NOISE_RATIO)
    data_metadata["n_bkps"].append(len(gt_bkps)-1)
    save_data(G, signal, gt_bkps, subdir)

# storing metadata
data_metadata = turn_all_list_of_dict_into_str(data_metadata)
create_parent_and_dump_json(data_dir, "metadata.json", data_metadata, indent=4)

In [ ]:
# initialization
DATE = None
if DATE :
    date = "2024-06-18_17-22-19"
    data_dir = "data/synthetic_data/exp_B_totally_out_of_hypothesis/B_1/" + date

################################################################
results_dir = "results/synthetic/D_in_hypo_diag_white_noise/D_5/" + NAME
data_dir = data_dir
################################################################

data_stats = open_json(f"{data_dir}/metadata.json")
data_stats["bkps"] = {}
data_stats["data_path"] = data_dir

# output formatting
statio_results = {}
normal_results = {}

subfold_list = [int(subdir) for subdir in os.listdir(data_dir) if '.' not in subdir]
subfold_list.sort()
for exp_id in tqdm(subfold_list, desc='Running experiment...'):

    exp_id = str(exp_id) # just for compatibility

    # loading data
    subdir = f"{data_dir}/{exp_id}"
    G, signal, gt_bkps = read_data(subdir)
    min_size = get_min_size_for_hyp(G.number_of_nodes(), hyp=data_stats["bkps_gap_hyp"])
    data_stats["bkps"][exp_id] = gt_bkps

    # prediction with standard normal cost
    t1 = time.perf_counter()
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        normal_cost = rpt.costs.CostNormal()
        algo_normal = rpt.Dynp(custom_cost=normal_cost, jump=1, min_size=min_size).fit(signal)
        normal_bkps = algo_normal.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    normal_results[exp_id] = {}
    normal_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    normal_results[exp_id]["pred"] = normal_bkps

    # prediction with stationary normal cost
    t1 = time.perf_counter()
    statio_cost = CostGraphStatioNormal(nx.laplacian_matrix(G).toarray())
    algo_statio = rpt.Dynp(custom_cost=statio_cost, jump=1, min_size=min_size).fit(signal)
    statio_bkps = algo_statio.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    statio_results[exp_id] = {}
    statio_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    statio_results[exp_id]["pred"] = statio_bkps

# storing
create_parent_and_dump_json(results_dir, "data_stats.json", turn_all_list_of_dict_into_str(data_stats), indent=4)
create_parent_and_dump_json(results_dir, "statio_pred.json", turn_all_list_of_dict_into_str(statio_results), indent=4)
create_parent_and_dump_json(results_dir, "normal_pred.json", turn_all_list_of_dict_into_str(normal_results), indent=4)


In [ ]:
PRECI_RECALL_MARGIN = 2

#############################################################
pred_dir = results_dir
#############################################################

# initialization
data_stats = open_json(f"{results_dir}/data_stats.json")
statio_pred_dic = open_json(f"{results_dir}/statio_pred.json")
normal_pred_dic = open_json(f"{results_dir}/normal_pred.json")
gt_bkps_dic = data_stats.pop('bkps')

# output formatting
metrics_dic = {}
metrics_dic["pred_path"] = results_dir
metrics_dic["hyper-parameters"] = data_stats
metrics_dic["hyper-parameters"]["metrics_margin"] = PRECI_RECALL_MARGIN

statio_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}
normal_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}

for exp_id in gt_bkps_dic.keys():
    # compute metrics
    normal_pred_bkps = turn_str_of_list_into_list_of_int(normal_pred_dic[exp_id]["pred"])
    statio_pred_bkps = turn_str_of_list_into_list_of_int(statio_pred_dic[exp_id]["pred"])
    gt_bkps = turn_str_of_list_into_list_of_int(gt_bkps_dic[exp_id])
    compute_and_update_metrics(gt_bkps, normal_pred_bkps, normal_results, PRECI_RECALL_MARGIN)
    compute_and_update_metrics(gt_bkps, statio_pred_bkps, statio_results, PRECI_RECALL_MARGIN)
    # add time values
    normal_results["time"]["raw"].append(normal_pred_dic[exp_id]["time"])
    statio_results["time"]["raw"].append(statio_pred_dic[exp_id]["time"])

# results post-precessing and saving
full_results = {"statio normal cost": statio_results, "normal cost": normal_results}
full_results = compute_and_add_stat_on_metrics(full_results)
full_results = turn_all_list_of_dict_into_str(full_results)
create_parent_and_dump_json(pred_dir, f'metrics_{PRECI_RECALL_MARGIN}.json', full_results, indent=4)

In [ ]:
##### INITIALIZATION
PRECI_RECALL_MARGIN = 2
res_folder_list = [
    "results/synthetic/D_in_hypo_diag_white_noise/D_5/min_gap_cons_non_weighted_geo_snr1",
    "results/synthetic/D_in_hypo_diag_white_noise/D_5/min_gap_cons_non_weighted_geo_snr2",
    "results/synthetic/D_in_hypo_diag_white_noise/D_5/min_gap_cons_non_weighted_geo_snr4",
]
abscissa =  ["SNR = 1", "SNR = 2", "SNR = 4"]
cost_func_keys = ["statio normal cost", "normal cost"]
colors_per_cost_func = {"statio normal cost": "darkorange", "normal cost": "dodgerblue"}
metrics_keys = ["precision", "hausdorff"]
res_dic = get_res_per_metric_per_cost_func(res_folder_list, cost_func_keys, metrics_keys, preci_margin=PRECI_RECALL_MARGIN)

##### PLOTTING
fig, axes = plt.subplots(1, (len(metrics_keys)), figsize=(6*len(metrics_keys), 5)) #layout='constrained')
# precision
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "precision", axes[0], to_label=True)
axes[0].set_title('Precision per cost function')
axes[0].set_ylim(bottom=0)
# haussdorff
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "hausdorff", axes[1],  to_label=False)
axes[1].set_title('Haussdorff per cost function')
axes[1].set_ylim(bottom=0)
axes[1].set_ylim(top=400)

N_min = 40
N_max = 50
graph_type = "non-weighted geographic"
gap_hypothesis = "minimal"
N_iter = 100
T = 1000

fig.suptitle(f"T={T}, N_min={N_min}, N_max={N_max}, {graph_type} graphs, {N_iter} iterations, margin={PRECI_RECALL_MARGIN}")
fig.legend(loc=(0.3, 0), ncols=len(cost_func_keys))

save_path = "images/results/synthetic/D_diag_white_noise/SNR"
if not os.path.exists(save_path):
    Path(save_path).mkdir(parents=True, exist_ok=False)
name = 'geo_non_weighted.png'
fig.savefig(os.path.join(save_path, name))

### E. Robustness with respect to (temporaly) independent additive plain white noise

### F. Robustness with respect to the graph structure: modification of the connectivity

In this experiment, we do not generate the signal $(y_t)_{1 \leq t \leq T}$ with the graph $G$ that is used to compute the cost function. Instead, we rather utilize the laplacian matrix $L_{noisy}$ of a noisy version $G_{noisy}$ of the original graph $G$.

Let denote $M = |E|$ the number of edges in $G$ and $\eta_{edge}$ the proportion of edges that we modify. We randomly remove $M_{remove} = \lfloor M * \eta_{edge} / 2 \rfloor$ from the set E and we randomly add $M_{add} = \lfloor M * \eta_{edge} / 2 \rfloor$ to $E$. In both cases, we select the edges randomly, but we always check that the resulting noisy graph $G_{noisy}$ still has the same number of nodes as $G$.

In [ ]:
def modify_graph_connectivity(G:nx.Graph, edge_prop, graph_rng:np.random.Generator):
    # initialization
    edges = [e for e in G.edges()]
    n_edges = len(edges)
    n_nodes = G.number_of_nodes()
    n_modif_edges = int(n_edges * edge_prop)
    # removing some random edges
    edge_ids_to_remove = graph_rng.integers(low=0, high=n_edges, size=n_modif_edges//2)
    edges_to_keep = [e for i, e in enumerate(edges) if i not in edge_ids_to_remove]
    # adding some edges
    new_edges_to_add = []
    while len(new_edges_to_add) < n_modif_edges//2 :
        new_edge = tuple(graph_rng.integers(low=0, high=n_nodes, size=2))
        if new_edge[0] != new_edge[1]:
            if new_edge not in new_edges_to_add:
                new_edges_to_add.append(new_edge)
    edges_new_graph = edges_to_keep + new_edges_to_add
    new_G = nx.from_edgelist(edges_new_graph)
    if new_G.number_of_nodes() == n_nodes:
        return nx.from_edgelist(edges_new_graph)
    else:
        return modify_graph_connectivity(G, edge_prop, graph_rng)

In [ ]:
nx_graph_seed = 1
graph_seed = 2
signal_seed = 5

signal_rng = np.random.default_rng(seed=signal_seed)
graph_rng = np.random.default_rng(seed=graph_seed)

G, coord = generate_random_weighted_geographic_graph(graph_rng, min_n_nodes=80, max_n_nodes=110, dist_threshold=0.2)
G_modif = modify_graph_connectivity(G, 0.1, graph_rng)
bkps, s = generate_rd_signal_in_hyp_with_max_tries(G_modif, signal_rng, n_bkps_max=10, n_samples=G.number_of_nodes()**2, hyp='minimal', diag_cov_max=1)

original_edges = set([(min(e), max(e)) for e in G.edges()])
new_edges = set([(min(e), max(e)) for e in G_modif.edges()])

added = new_edges.difference(original_edges)
withdrawn  = original_edges.difference(new_edges)
same = new_edges.intersection(original_edges)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
# original graph
nx.draw_networkx_edges(G, pos=coord, edgelist=same, edge_color='k', ax=axes[0])
nx.draw_networkx_edges(G, pos=coord, edgelist=withdrawn, edge_color='r', width=3, ax=axes[0])
nx.draw_networkx(G, with_labels=False, pos=coord, node_size=50, ax=axes[0], edgelist = [])
# modified graph
nx.draw_networkx_edges(G_modif, pos=coord, edgelist=same, edge_color='k')
nx.draw_networkx_edges(G_modif, pos=coord, edgelist=added, width=3, edge_color='g')
nx.draw_networkx(G_modif, with_labels=False, pos=coord, node_size=50, ax=axes[1], edgelist = [])

In [ ]:
N_EXP = 100
NX_GRAPH_SEED = 1
GRAPH_SEED = 2
SIGNAL_SEED = 3
MIN_N_NODES = 40
MAX_N_NODES = 50
DIST_THRESHOLD = 0.3
DIAG_COV_MAX = 1
N_SAMPLES = 1000
MAX_N_BKPS = 10
EDGE_PROP_MODIF = 0.05
BKPS_GAP_CONSTRAINT: seg_length = "minimal" 

###################################################################
NAME = "min_gap_cons_non_weighted_geo_edgeprop005"
data_dir = "data/synthetic_data/exp_F_in_hypo_graph_connec_modif/F_5/" + NAME
desc = ['Minimal gap constraint, using non-weighted geographical graphs, for testing with a proportion of edges being modified.', 'The signal is generated within the model hypothesis but based on the Laplacian of a modified graph and we use the orginal graph to compute the cost function (that is, we use a noisy version of the graph to compute the cost function).', 'Purpose: check if the cost functions are sensitive to a noisy graph observation']
## FUNCTIONS USED FOR GRAPH GENERATION
###################################################################

# initialization
signal_rng = np.random.default_rng(seed=SIGNAL_SEED)
graph_rng = np.random.default_rng(seed=GRAPH_SEED)
data_metadata = {"description": desc, "commit hash":get_git_head_short_hash(), "graph_func": generate_random_geographic_graph.__name__, "signal_func": generate_rd_signal_in_hyp_with_max_tries.__name__ + " + " + modify_graph_connectivity.__name__ + ": the signal is generated with cov mat coming from the modified graph, and the cost function will be instantiated with the original graph", "n_iter": N_EXP, "graph_seed": GRAPH_SEED, "nx_graph_seed": NX_GRAPH_SEED, "signal_seed": SIGNAL_SEED, "n_samples": N_SAMPLES, "min_n_nodes": MIN_N_NODES, "dist_threshold": DIST_THRESHOLD, "max_n_nodes": MAX_N_NODES, "max_n_bkps": MAX_N_BKPS, "bkps_gap_hyp": BKPS_GAP_CONSTRAINT, "diag_cov_max": DIAG_COV_MAX, "modif_edges_prop": EDGE_PROP_MODIF, "n_nodes": [], 'n_bkps': []}

# data generation and saving
for exp_id in range(N_EXP):
    subdir = f"{data_dir}/{exp_id}"
    G, _ = generate_random_geographic_graph(graph_rng, min_n_nodes=MIN_N_NODES, max_n_nodes=MAX_N_NODES, dist_threshold=DIST_THRESHOLD)
    G_modif = modify_graph_connectivity(G, EDGE_PROP_MODIF, graph_rng)
    n_nodes = int(G.number_of_nodes())
    data_metadata["n_nodes"].append(n_nodes)
    gt_bkps, signal = generate_rd_signal_in_hyp_with_max_tries(G_modif, signal_rng, n_bkps_max=MAX_N_BKPS, hyp=BKPS_GAP_CONSTRAINT, n_samples=N_SAMPLES, diag_cov_max=DIAG_COV_MAX)
    data_metadata["n_bkps"].append(len(gt_bkps)-1)
    save_data(G, signal, gt_bkps, subdir)
    adj_mat_modif = nx.to_numpy_array(G_modif)
    with open(f'{subdir}/modif_graph_adj_mat.npy', 'wb+') as nx_f:
        np.save(nx_f, adj_mat_modif, allow_pickle=False)

# storing metadata
data_metadata = turn_all_list_of_dict_into_str(data_metadata)
create_parent_and_dump_json(data_dir, "metadata.json", data_metadata, indent=4)

In [ ]:
# initialization
DATE = None
if DATE :
    date = "2024-06-18_17-22-19"
    data_dir = "data/synthetic_data/exp_B_totally_out_of_hypothesis/B_1/" + date

################################################################
results_dir = "results/synthetic/F_in_hypo_graph_connec_modif/F_5/" + NAME
data_dir = data_dir
################################################################

data_stats = open_json(f"{data_dir}/metadata.json")
data_stats["bkps"] = {}
data_stats["data_path"] = data_dir

# output formatting
statio_results = {}
normal_results = {}

subfold_list = [int(subdir) for subdir in os.listdir(data_dir) if '.' not in subdir]
subfold_list.sort()
for exp_id in tqdm(subfold_list, desc='Running experiment...'):

    exp_id = str(exp_id) # just for compatibility

    # loading data
    subdir = f"{data_dir}/{exp_id}"
    G, signal, gt_bkps = read_data(subdir)
    min_size = get_min_size_for_hyp(G.number_of_nodes(), hyp=data_stats["bkps_gap_hyp"])
    data_stats["bkps"][exp_id] = gt_bkps

    # prediction with standard normal cost
    t1 = time.perf_counter()
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        normal_cost = rpt.costs.CostNormal()
        algo_normal = rpt.Dynp(custom_cost=normal_cost, jump=1, min_size=min_size).fit(signal)
        normal_bkps = algo_normal.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    normal_results[exp_id] = {}
    normal_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    normal_results[exp_id]["pred"] = normal_bkps

    # prediction with stationary normal cost
    t1 = time.perf_counter()
    statio_cost = CostGraphStatioNormal(nx.laplacian_matrix(G).toarray())
    algo_statio = rpt.Dynp(custom_cost=statio_cost, jump=1, min_size=min_size).fit(signal)
    statio_bkps = algo_statio.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    statio_results[exp_id] = {}
    statio_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    statio_results[exp_id]["pred"] = statio_bkps

# storing
create_parent_and_dump_json(results_dir, "data_stats.json", turn_all_list_of_dict_into_str(data_stats), indent=4)
create_parent_and_dump_json(results_dir, "statio_pred.json", turn_all_list_of_dict_into_str(statio_results), indent=4)
create_parent_and_dump_json(results_dir, "normal_pred.json", turn_all_list_of_dict_into_str(normal_results), indent=4)


In [ ]:
PRECI_RECALL_MARGIN = 5

#############################################################
# pred_dir = results_dir
pred_dir = "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop002"
results_dir = pred_dir
#############################################################

# initialization
data_stats = open_json(f"{results_dir}/data_stats.json")
statio_pred_dic = open_json(f"{results_dir}/statio_pred.json")
normal_pred_dic = open_json(f"{results_dir}/normal_pred.json")
gt_bkps_dic = data_stats.pop('bkps')

# output formatting
metrics_dic = {}
metrics_dic["pred_path"] = results_dir
metrics_dic["hyper-parameters"] = data_stats
metrics_dic["hyper-parameters"]["metrics_margin"] = PRECI_RECALL_MARGIN

statio_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []},  "hausdorff": {'raw': []}, "time": {"raw": []}}
normal_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}

for exp_id in gt_bkps_dic.keys():
    # compute metrics
    normal_pred_bkps = turn_str_of_list_into_list_of_int(normal_pred_dic[exp_id]["pred"])
    statio_pred_bkps = turn_str_of_list_into_list_of_int(statio_pred_dic[exp_id]["pred"])
    gt_bkps = turn_str_of_list_into_list_of_int(gt_bkps_dic[exp_id])
    compute_and_update_metrics(gt_bkps, normal_pred_bkps, normal_results, PRECI_RECALL_MARGIN)
    compute_and_update_metrics(gt_bkps, statio_pred_bkps, statio_results, PRECI_RECALL_MARGIN)
    # add time values
    normal_results["time"]["raw"].append(normal_pred_dic[exp_id]["time"])
    statio_results["time"]["raw"].append(statio_pred_dic[exp_id]["time"])

# results post-precessing and saving
full_results = {"statio normal cost": statio_results, "normal cost": normal_results}
full_results = compute_and_add_stat_on_metrics(full_results)
full_results = turn_all_list_of_dict_into_str(full_results)
create_parent_and_dump_json(pred_dir, f'metrics_{PRECI_RECALL_MARGIN}.json', full_results, indent=4)

In [ ]:
##### INITIALIZATION
PRECI_RECALL_MARGIN = 5
res_folder_list = [
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop002",
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop005",
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop01",
    "results/synthetic/F_in_hypo_graph_connec_modif/F_5/min_gap_cons_non_weighted_geo_edgeprop02",
]
abscissa =  ["eta = 0.02", "eta = 0.05", "eta = 0.1", "eta = 0.2"]
cost_func_keys = ["statio normal cost", "normal cost"]
colors_per_cost_func = {"statio normal cost": "darkorange", "normal cost": "dodgerblue"}
metrics_keys = ["precision", "hausdorff"]
res_dic = get_res_per_metric_per_cost_func(res_folder_list, cost_func_keys, metrics_keys, preci_margin=PRECI_RECALL_MARGIN)

##### PLOTTING
fig, axes = plt.subplots(1, (len(metrics_keys)), figsize=(6*len(metrics_keys), 5)) #layout='constrained')
# precision
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "precision", axes[0], to_label=True)
axes[0].set_title('Precision per cost function')
axes[0].set_ylim(bottom=0)
# haussdorff
plot_bar_and_scatter(cost_func_keys, res_dic, abscissa, colors_per_cost_func, "hausdorff", axes[1],  to_label=False)
axes[1].set_title('Haussdorff per cost function')
axes[1].set_ylim(bottom=0)
axes[1].set_ylim(top=300)

N_min = 40
N_max = 50
graph_type = "non-weighted geographic"
gap_hypothesis = "minimal"
N_iter = 100
T = 1000

fig.suptitle(f"T={T}, N_min={N_min}, N_max={N_max}, {graph_type} graphs, {N_iter} iterations, margin={PRECI_RECALL_MARGIN}")
fig.legend(loc=(0.3, 0), ncols=len(cost_func_keys))

save_path = "images/results/synthetic/F_noisy_grap/edge_proportion"
if not os.path.exists(save_path):
    Path(save_path).mkdir(parents=True, exist_ok=False)
name = 'geo_non_weighted.png'
fig.savefig(os.path.join(save_path, name))

## Experimental setting v.1

### Data writing and reading

In [ ]:
def save_graph(G:nx.Graph, path):
    if os.path.exists(path):
        raise FileExistsError
    # save graph
    adj_mat = nx.to_numpy_array(G)
    with open(path, 'wb+') as nx_f:
        np.save(nx_f, adj_mat, allow_pickle=False)

def save_signal_and_bkps(signal:np.ndarray, bkps:List[int], dir, name):
    path = os.path.join(dir, name)
    if os.path.exists(f"{path}_signal.npy"):
        raise FileExistsError
    # save signal
    with open(f"{path}_signal.npy", 'wb+') as np_f:
        np.save(np_f, signal, allow_pickle=False)
    # save bkps
    bkps_str = [int(bkp) for bkp in bkps]
    with open(f"{path}_bkps.json", 'w+') as f:
        json.dump(bkps_str, f, indent=None)

In [ ]:
def read_data_1(graph_path: str, signal_path: str, exp_id: int):
    adj_mat = np.load(f"{graph_path}/{exp_id}_mat_adj.npy", allow_pickle=False)
    G = nx.from_numpy_array(adj_mat)
    signal = np.load(f"{signal_path}/{exp_id}_signal.npy", allow_pickle=False)
    bkps = open_json(f"{signal_path}/{exp_id}_bkps.json")
    return G, signal, bkps

def load_data(graph_path: str, signal_path: str, exp_id: int, hyp:seg_length):
    G, signal, bkps = read_data_1(graph_path, signal_path, exp_id)
    min_size = get_min_size_for_hyp(G.number_of_nodes(), hyp=hyp)
    return G, signal, bkps, min_size

### Data generation and vizualization

#### Graph generation

In [ ]:
def generate_random_er_graphs_fixed_nodes_nb(params_rng, nx_graph_seed, n_nodes, target_deg, bandwidth_coef):
    min_edge_p = (1- bandwidth_coef) * target_deg / (n_nodes -1)
    max_edge_p = (1+ bandwidth_coef) * target_deg / (n_nodes -1)
    edge_p = min_edge_p + (max_edge_p-min_edge_p) * params_rng.random()
    G = nx.erdos_renyi_graph(n=n_nodes, p=edge_p, seed=nx_graph_seed)
    params = {"edge_prob": edge_p}
    return G, params

In [ ]:
graph_seed = 0
params_seed = 1
fig, axes = plt.subplots(1, 5, figsize=(4*5, 3))
params_rng = np.random.default_rng(seed=params_seed)
n_nodes = 40
target_deg = 10
bandwidth = 0.4

av_deg = []
for _ in range(5):
    G, a = generate_random_er_graphs_fixed_nodes_nb(params_rng, _, n_nodes, target_deg, bandwidth)
    coord = nx.spring_layout(G, seed=0)
    nx.draw_networkx(G, with_labels=False, pos=coord, node_size=50, ax=axes[_])
    degrees = [val for (node, val) in G.degree()]
    av_deg.append(sum(degrees)/n_nodes)
print(av_deg)

In [ ]:
def generate_random_geographic_graph_with_gauss_kernel(params_rng, n_nodes, target_degree):
    # generation of the nodes coordinates
    nodes_coord = params_rng.random(size=(n_nodes, 2))
    # computation of the threshold for the exponential adjacency matrix
    dist_mat_condensed = pdist(nodes_coord, metric='euclidean')
    sigma = np.median(dist_mat_condensed)  # median heuristic
    expsim_condensed = np.exp(-(dist_mat_condensed**2) / (sigma**2))
    # ordering the values to find the right threshold
    ordered_exp_sim = np.sort(expsim_condensed)
    n_edge_for_target_deg = target_degree*n_nodes//2
    threshold = ordered_exp_sim[-n_edge_for_target_deg+1]
    # creating the corresponding adjacency matrix and graph
    adj_mat_condensed = np.where(expsim_condensed > threshold, expsim_condensed, 0.0)
    adj_mat = squareform(adj_mat_condensed)
    G = nx.Graph(adj_mat)
    return G, nodes_coord

In [ ]:
params_seed = 1
fig, axes = plt.subplots(1, 5, figsize=(4*5, 3))
params_rng = np.random.default_rng(seed=params_seed)
n_nodes = 80
target_deg = 10
av_deg = []
to_plot_coords = []
for _ in range(5):
    G, coord = generate_random_geographic_graph_with_gauss_kernel(params_rng,  n_nodes=n_nodes, target_degree=target_deg)
    nx.draw_networkx(G, with_labels=False, pos=coord, node_size=50, ax=axes[_])
    degrees = [val for (node, val) in G.degree()]
    av_deg.append(sum(degrees)/n_nodes)
    to_plot_coords.append(coord)
print(av_deg)

In [ ]:
from libpysal.weights import KNN

def generate_KNN_random_geographic_graph(params_rng, n_nodes, K):
    # generation of the nodes coordinates 
    nodes_coord = params_rng.random(size=(n_nodes, 2))
    # computation of the KNN adjacency matrix
    knn_weights = KNN.from_array(nodes_coord, K)
    # build the graph
    G_directed = knn_weights.to_networkx()
    G = G_directed.to_undirected()
    return G, nodes_coord

In [ ]:
params_seed = 1
fig, axes = plt.subplots(1, 5, figsize=(4*5, 3))
params_rng = np.random.default_rng(seed=params_seed)
n_nodes = 80
K = 8
av_deg = []
for _ in range(5):
    G, coord = generate_KNN_random_geographic_graph(params_rng,  n_nodes=n_nodes, K=K)
    nx.draw_networkx(G, with_labels=False, pos=coord, node_size=50, ax=axes[_])
    degrees = [val for (node, val) in G.degree()]
    av_deg.append(sum(degrees)/n_nodes)
print(av_deg)

In [ ]:
N_EXP = 1000
GRAPH_SEED = 1
N_NODES = 100
TARGET_DEGREE = 10
K_NEIGHBOUR = 8
ER_BANDWIDTH = 0.4

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
graph_rng = np.random.default_rng(seed=GRAPH_SEED)

# logging
NAME = f"ER_{N_NODES}_nodes_deg_{TARGET_DEGREE}_bandwidth_{ER_BANDWIDTH}"
data_dir = "data_1/graphs/clean_ER_with_bandwidth/" + NAME
graphs_desc = "ER graphs with fixed number of nodes. The edge probability is randomly drawn based on the target degree but also using a bandwidth parameter to allow for more diversity. Otherwise, for a given number of nodes and edge probability, the generated graphs would always be the same based on the networkx implementation."
graph_gen_func = lambda rng, graph_seed : generate_random_er_graphs_fixed_nodes_nb(rng, graph_seed, n_nodes=N_NODES, target_deg=TARGET_DEGREE, bandwidth_coef=ER_BANDWIDTH)
graphs_metadata = {"datetime": now, "description": graphs_desc, "commit hash": get_git_head_short_hash(), "graph_func": generate_random_er_graphs_fixed_nodes_nb.__name__, "graph_seed": GRAPH_SEED, "nx_graph_seed": "index", "n_nodes": N_NODES, "target_degree": TARGET_DEGREE, "bandwidth coefficient": ER_BANDWIDTH}

# output formatting
Path(data_dir).mkdir(parents=True, exist_ok=False)
graphs_metadata = turn_all_list_of_dict_into_str(graphs_metadata)
create_parent_and_dump_json(data_dir, "00_graphs_metadata.json", graphs_metadata, indent=4)

# graph generation
for exp_id in range(N_EXP):
    G, coords = graph_gen_func(graph_rng, exp_id)
    save_graph(G, f"{data_dir}/{exp_id}_mat_adj.npy")

#### Signal and bkps generation

In [ ]:
def generate_rd_signal_in_hyp(G:nx.Graph, signal_rng:np.random.Generator, hyp:seg_length, n_samples:int, diag_cov_max):
    # randomly draw a set of admissible change points
    n_dims = G.number_of_nodes()
    min_size = get_min_size_for_hyp(n_dims=n_dims, hyp=hyp)
    bkps = draw_bkps_with_gap_constraint(n_samples=n_samples, bkps_gap=min_size, bkps_rng=signal_rng, n_bkps_max=n_samples)
    # generate the signal
    _, eigvects = eigh(nx.laplacian_matrix(G).toarray())
    signal_gen_func = lambda size: generate_gaus_signal_with_cov_diag_in_basis(n_dims, size, eigvects, signal_rng, diag_cov_max)
    signal = signal_gen_func(bkps[0])
    # add each sub-segment
    for i in range(1, len(bkps)):
        sub_signal = signal_gen_func(bkps[i] - bkps[i-1])
        signal = np.concatenate([signal, sub_signal], axis=0)
    return bkps, signal.astype(np.float64)

In [ ]:
def generate_rd_signal_in_hyp_with_fixed_min_size(G:nx.Graph, signal_rng:np.random.Generator, hyp:seg_length, n_samples:int, min_size_coef:int, diag_cov_max):
    # randomly draw a set of admissible change points
    n_dims = G.number_of_nodes()
    min_size = int(min_size_coef * get_min_size_for_hyp(n_dims=n_dims, hyp=hyp))
    bkps = draw_fixed_nb_bkps_with_gap_constraint(n_samples=n_samples, bkps_gap=min_size, bkps_rng=signal_rng)
    # generate the signal
    _, eigvects = eigh(nx.laplacian_matrix(G).toarray())
    signal_gen_func = lambda size: generate_gaus_signal_with_cov_diag_in_basis(n_dims, size, eigvects, signal_rng, diag_cov_max)
    signal = signal_gen_func(bkps[0])
    # add each sub-segment
    for i in range(1, len(bkps)):
        sub_signal = signal_gen_func(bkps[i] - bkps[i-1])
        signal = np.concatenate([signal, sub_signal], axis=0)
    return bkps, signal.astype(np.float64)

In [ ]:
GRAPH_FOLDER = "data_1/graphs/clean_ER_with_bandwidth"
GRAPH_FOLDER_NAME = "ER_40_nodes_deg_10_bandwidth_0.4"
GRAPH_PATH = os.path.join(GRAPH_FOLDER, GRAPH_FOLDER_NAME)
SIGNAL_SEED = 3
DIAG_COV_MAX = 1
MIN_SEGMENT_LENGTH_COEF = 1.2
SNR = 2
N_SAMPLES = 1000
BKPS_GAP_CONSTRAINT: seg_length = "minimal" 

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
signal_rng = np.random.default_rng(seed=SIGNAL_SEED)
sigma_noise = DIAG_COV_MAX / ( 10**(SNR / 10) )

# logging
NAME = GRAPH_FOLDER_NAME
data_dir = f"data_1/signal/within_hyp/varying_segment_length/minimal_x{MIN_SEGMENT_LENGTH_COEF}_" + NAME
signal_desc = "'Data verifying the two hypothesis, with a given number of bkps fixed by a coefficient applied to the minimal segment length."
signal_gen_func = lambda G : generate_rd_signal_in_hyp_with_fixed_min_size(G, signal_rng, hyp=BKPS_GAP_CONSTRAINT, n_samples=N_SAMPLES, min_size_coef=MIN_SEGMENT_LENGTH_COEF, diag_cov_max=DIAG_COV_MAX)
signal_metadata = {"datetime": now, "description": signal_desc, "commit hash": get_git_head_short_hash(), "graph_folder": GRAPH_PATH, "signal_seed": SIGNAL_SEED, "signal_gen_function": generate_rd_signal_in_hyp.__name__, "n_samples": N_SAMPLES, "diag_cov_max": DIAG_COV_MAX, "min_size_coef": MIN_SEGMENT_LENGTH_COEF, "bkps_gap_hyp": BKPS_GAP_CONSTRAINT}

# output formatting
Path(data_dir).mkdir(parents=True, exist_ok=False)
signal_metadata = turn_all_list_of_dict_into_str(signal_metadata)
create_parent_and_dump_json(data_dir, "00_signal_metadata.json", signal_metadata, indent=4)

# signal generation
for exp_id in range(len(os.listdir(GRAPH_PATH)) - 1):
    adj_mat = np.load(f"{GRAPH_PATH}/{exp_id}_mat_adj.npy", allow_pickle=False)
    G = nx.from_numpy_array(adj_mat)
    bkps, signal = signal_gen_func(G)
    save_signal_and_bkps(signal, bkps, data_dir, str(exp_id))

### Running experiment

In [ ]:
def run_statio_normal_cost(G: nx.Graph, signal: np.ndarray, gt_bkps: List[int], statio_results: dict):
    # running CPD algorithm
    t1 = time.perf_counter()
    statio_cost = CostGraphStatioNormal(nx.laplacian_matrix(G).toarray())
    algo_statio = rpt.Dynp(custom_cost=statio_cost, jump=1, min_size=min_size).fit(signal)
    statio_bkps = algo_statio.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    # logging
    statio_results[exp_id] = {}
    statio_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    statio_results[exp_id]["pred"] = statio_bkps
    statio_results[exp_id]["gt"] = gt_bkps
    statio_results[exp_id]["n_bkps"] = len(gt_bkps)-1

In [ ]:
def run_standard_mle_normal_cost(signal: np.ndarray, gt_bkps: List[int], normal_results: dict):
    # running CPD algorithm
    t1 = time.perf_counter()
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        normal_cost = rpt.costs.CostNormal()
        algo_normal = rpt.Dynp(custom_cost=normal_cost, jump=1, min_size=min_size).fit(signal)
        normal_bkps = algo_normal.predict(n_bkps=len(gt_bkps)-1)
    t2 = time.perf_counter()
    # logging
    normal_results[exp_id] = {}
    normal_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    normal_results[exp_id]["pred"] = normal_bkps
    normal_results[exp_id]["gt"] = gt_bkps
    normal_results[exp_id]["n_bkps"] = len(gt_bkps)-1

#### Numba version

- do not use lists, fill-in pre-allocated np.ndarray
- use functions rather than classes

In [ ]:
def init_station_normal_cost(signal, graph_laplacian_mat):
    # computation of the graph fourier transform
    _, eigvects = eigh(graph_laplacian_mat)
    gft =  signal @ eigvects # equals signal.dot(eigvects) = eigvects.T.dot(signal.T).T
    gft_mean = np.mean(gft, axis=0)
    # computation of the per-segment cost utils
    gft_square_cumsum = np.concatenate([np.zeros((1, signal.shape[1])), np.cumsum((gft - gft_mean[None, :])**2, axis=0)], axis=0)
    return gft_square_cumsum.astype(np.float64)

@njit
def numba_statio_cost_func(start, end, gft_square_cumsum):
    sub_square_sum = gft_square_cumsum[end] - gft_square_cumsum[start]
    return np.float64(end  - start) * np.sum(np.log(sub_square_sum / (end - start)), dtype=np.float64)

@njit
def numba_cpd_dynprog_statio_cost(n_bkps:int, min_size:int, data: np.ndarray) -> List[np.int64]:
    n_samples = data.shape[0]
    # if no bkp to find
    if n_bkps == 0:
        return np.array([1000], dtype=np.int64)
    # full partitions costs
    full_part_cost = np.inf * np.ones((n_bkps, n_samples, n_samples), dtype=np.float64)
    # compute the segment cost with no bpk, for admissible segment only
    for start in range(0, n_samples-min_size):
        # until n_samples + 1 because the call to cost_function(start, end, data) computes over [start, end-1]
        for end in range(start+min_size, n_samples):  
            full_part_cost[0, start, end] = numba_statio_cost_func(start, end, data)
    # compute the cost of the possible higher order partitions 
    for bkp_order in range(1, n_bkps):
        min_multi_seg_length = (bkp_order + 1) * min_size
        for start in range(0, n_samples-min_multi_seg_length):
            for end in range(start + min_multi_seg_length, n_samples):
                min_size_left_seg = min_multi_seg_length - min_size
                full_part_cost[bkp_order, start, end] = np.min(full_part_cost[bkp_order-1, start, start+min_size_left_seg:end-min_size+1] + full_part_cost[0, start+min_size_left_seg:end-min_size+1, end])
    # successively pick the bkps from the right-end of the whole signal
    bkps = np.int64(n_samples-1) * np.ones(n_bkps+1, dtype=np.int64)
    for bkp_id in range(n_bkps, 0, -1):
        min_multi_seg_length = np.int64(bkp_id * min_size) 
        bkp_right = bkps[bkp_id]
        bkp_left = min_multi_seg_length + np.argmin(full_part_cost[bkp_id-1, 0, min_multi_seg_length:bkp_right-min_size+1] + full_part_cost[0, min_multi_seg_length:bkp_right-min_size+1, bkp_right])
        bkps[bkp_id-1] = bkp_left
    return bkps

def run_numba_statio_normal_cost(G: nx.Graph, signal: np.ndarray, gt_bkps: List[int], statio_results: dict):
    # running CPD algorithm
    t1 = time.perf_counter()
    graph_lapl_mat = nx.laplacian_matrix(G).toarray().astype(np.float64)
    gft_square_cumsum = init_station_normal_cost(signal, graph_lapl_mat)
    statio_bkps = numba_cpd_dynprog_statio_cost(len(gt_bkps)-1, signal.shape[1], gft_square_cumsum)
    statio_bkps = [int(bkp) for bkp in statio_bkps]
    t2 = time.perf_counter()
    # logging
    statio_results[exp_id] = {}
    statio_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    statio_results[exp_id]["pred"] = statio_bkps
    statio_results[exp_id]["gt"] = gt_bkps
    statio_results[exp_id]["n_bkps"] = len(gt_bkps)-1

In [ ]:
from numpy.linalg import slogdet

@njit
def standard_normal_cost_func(start, end, signal):
    sub = signal[start:end]
    cov = np.cov(sub.T)
    cov += 1e-6 * np.eye(signal.shape[1])
    _, val = slogdet(cov)
    return np.float64(val * (end - start))

@njit
def numba_cpd_dynprog_mle_standard_cost(n_bkps:int, min_size:int, signal: np.ndarray) -> List[np.int64]:
    n_samples = signal.shape[0]
    # if no bkp to find
    if n_bkps == 0:
        return np.array([1000], dtype=np.int64)
    # full partitions costs
    full_part_cost = np.inf * np.ones((n_bkps, n_samples, n_samples), dtype=np.float64)
    # compute the segment cost with no bpk, for admissible segment only
    for start in range(0, n_samples-min_size+1):
        # until n_samples + 1 because the call to cost_function(start, end, data) computes over [start, end-1]
        for end in range(start+min_size, n_samples+1):  
            full_part_cost[0, start, end] = standard_normal_cost_func(start, end, signal)
    # compute the cost of the possible higher order partitions 
    for bkp_order in range(1, n_bkps):
        min_multi_seg_length = (bkp_order + 1) * min_size
        for start in range(0, n_samples-min_multi_seg_length):
            for end in range(start + min_multi_seg_length, n_samples):
                min_size_left_seg = min_multi_seg_length - min_size
                full_part_cost[bkp_order, start, end] = np.min(full_part_cost[bkp_order-1, start, start+min_size_left_seg:end-min_size+1] + full_part_cost[0, start+min_size_left_seg:end-min_size+1, end])
    # successively pick the bkps from the right-end of the whole signal
    bkps = np.int64(n_samples) * np.ones(n_bkps+1, dtype=np.int64)
    for bkp_id in range(n_bkps, 0, -1):
        min_multi_seg_length = np.int64(bkp_id * min_size) 
        bkp_right = bkps[bkp_id]
        bkp_left = min_multi_seg_length + np.argmin(full_part_cost[bkp_id-1, 0, min_multi_seg_length:bkp_right-min_size+1] + full_part_cost[0, min_multi_seg_length:bkp_right-min_size+1, bkp_right])
        bkps[bkp_id-1] = bkp_left
    return bkps

def run_numba_standard_mle_normal_cost(signal: np.ndarray, gt_bkps: List[int], normal_results: dict):
    # running CPD algorithm
    t1 = time.perf_counter()
    normal_bkps = numba_cpd_dynprog_mle_standard_cost(len(gt_bkps) - 1, signal.shape[1], signal)
    normal_bkps = [int(bkp) for bkp in normal_bkps]
    t2 = time.perf_counter()
    # logging
    normal_results[exp_id] = {}
    normal_results[exp_id]["time"] = round(t2 - t1, ndigits=3)
    normal_results[exp_id]["pred"] = normal_bkps
    normal_results[exp_id]["gt"] = gt_bkps
    normal_results[exp_id]["n_bkps"] = len(gt_bkps)-1

In [ ]:
NAME = "exp_geo_100_nodes_av_deg_10"
GRAPH_PATH = "data_1/graphs/clean_exp_geo"
SIGNAL_PATH = "data_1/signal/within_hyp/segment_length_minimal"
MAX_ID_SUBSET = 80
RESULT_DIR = "results_1/synthetic/within_hypothesis"
RESULT_NAME = "80_exp"

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
final_name = NAME + "_" + RESULT_NAME
results_dir = os.path.join(RESULT_DIR, final_name)

# logging
graph_path = os.path.join(GRAPH_PATH, NAME)
signal_path = os.path.join(SIGNAL_PATH, NAME)
graph_metadata = open_json(f"{graph_path}/00_graphs_metadata.json")
signal_metadata = open_json(f"{signal_path}/00_signal_metadata.json")
seg_length_hyp = signal_metadata['bkps_gap_hyp']

exp_desc = "Within hypothesis, with no limit on the number of bkps. One of the goals is to evaluate the influence of the number of nodes."
experiment_metadata = {"datetime": now, "description": exp_desc, "commit hash": get_git_head_short_hash(), "graph folder": GRAPH_PATH, "graph metadata": graph_metadata, "signal folder": SIGNAL_PATH, "signal metadata": signal_metadata, "min segment length hypothesis": seg_length_hyp, "max id experiment subset": MAX_ID_SUBSET}

# output formatting
statio_results = {}
normal_results = {}

# running CPD algorithms
for exp_id in tqdm(range(MAX_ID_SUBSET), desc='Running experiment...'):
    exp_id = str(exp_id)
    G, signal, gt_bkps, min_size = load_data(graph_path, signal_path, exp_id, seg_length_hyp)
    run_numba_statio_normal_cost(G, signal, gt_bkps, statio_results)
    run_numba_standard_mle_normal_cost(signal, gt_bkps, normal_results)

create_parent_and_dump_json(results_dir, "experiment_metadata.json", turn_all_list_of_dict_into_str(experiment_metadata), indent=4)
create_parent_and_dump_json(results_dir, "statio_pred.json", turn_all_list_of_dict_into_str(statio_results), indent=4)
create_parent_and_dump_json(results_dir, "normal_pred.json", turn_all_list_of_dict_into_str(normal_results), indent=4)


### Results computation

In [ ]:
PRECI_RECALL_MARGIN = 2
pred_path = "results_1/synthetic/within_hypothesis/graph_types_and_number_of_nodes"
file_names = os.listdir(pred_path)
PRED_FOLDER = [os.path.join(pred_path, file_name) for file_name in file_names]

for pred_dir in PRED_FOLDER:

    # fetching predictions
    data_stats = open_json(f"{pred_dir}/experiment_metadata.json")
    statio_pred_dic = open_json(f"{pred_dir}/statio_pred.json")
    normal_pred_dic = open_json(f"{pred_dir}/normal_pred.json")
    assert list(normal_pred_dic.keys()) == list(statio_pred_dic.keys())

    # output formatting
    metrics_dic = {}
    metrics_dic["pred_path"] = pred_dir
    metrics_dic["hyper-parameters"] = data_stats
    metrics_dic["hyper-parameters"]["metrics_margin"] = PRECI_RECALL_MARGIN

    statio_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}
    normal_results = {"recall": {'raw': []}, "precision": {'raw': []}, "f1_score": {'raw': []}, "hausdorff": {'raw': []}, "time": {"raw": []}}

    for exp_id in normal_pred_dic.keys():
        # compute metrics
        normal_pred_bkps = turn_str_of_list_into_list_of_int(normal_pred_dic[exp_id]["pred"])
        statio_pred_bkps = turn_str_of_list_into_list_of_int(statio_pred_dic[exp_id]["pred"])
        gt_bkps = turn_str_of_list_into_list_of_int(normal_pred_dic[exp_id]["gt"])
        compute_and_update_metrics(gt_bkps, normal_pred_bkps, normal_results, PRECI_RECALL_MARGIN)
        compute_and_update_metrics(gt_bkps, statio_pred_bkps, statio_results, PRECI_RECALL_MARGIN)
        # add time values
        normal_results["time"]["raw"].append(normal_pred_dic[exp_id]["time"])
        statio_results["time"]["raw"].append(statio_pred_dic[exp_id]["time"])

    # results post-precessing and saving
    full_results = {"statio normal cost": statio_results, "normal cost": normal_results}
    full_results = compute_and_add_stat_on_metrics(full_results)
    full_results["metadata"] = metrics_dic
    full_results = turn_all_list_of_dict_into_str(full_results)
    create_parent_and_dump_json(pred_dir, f'metrics_{PRECI_RECALL_MARGIN}.json', full_results, indent=4)

### Plotting results

In [ ]:
##### INITIALIZATION
PRECI_RECALL_MARGIN = 2
pred_path = "results_1/synthetic/within_hypothesis/graph_types_and_number_of_nodes"
file_names = os.listdir(pred_path)
PRED_FOLDER = [os.path.join(pred_path, file_name) for file_name in file_names]

cost_func_keys = ["statio normal cost", "normal cost"]
markers = ['o', "s"]
linestyles = ["dashed", "dotted"]
metrics_keys = ["f1_score", "hausdorff"]
abscissa_pos =  [20, 40, 60, 80, 100]
graph_types = ["exp_geo", "KNN_geo", "ER"]
graph_colors = ['darkorange', 'dodgerblue', 'darkorchid']
shift = 3


##### PLOTTING
fig, axes = plt.subplots(1, (len(metrics_keys)), figsize=(7*len(metrics_keys), 5)) #, layout='constrained')

for i in range(len(graph_types)):

    # re-arrange the folder name so they have the right position for meaningful plotting
    graph_name = graph_types[i]
    res_folder_list = [folder_name for folder_name in PRED_FOLDER if graph_name in folder_name]
    res_folder_list.sort()
    res_folder_list.append(res_folder_list[0])
    res_folder_list.pop(0)
    res_dic = get_res_per_metric_per_cost_func(res_folder_list, cost_func_keys, metrics_keys, PRECI_RECALL_MARGIN)

    x_coords = [x + i*shift for x in abscissa_pos]
    plot_scatter_errorbar(x_coords, cost_func_keys, graph_colors[i], markers, linestyles, "f1_score", res_dic, ax=axes[0])
    axes[0].set_xlabel("Number of nodes")
    axes[0].set_title('F1 SCORE per cost function')
    plot_scatter_errorbar(x_coords, cost_func_keys, graph_colors[i], markers, linestyles, "hausdorff", res_dic, ax=axes[1])
    axes[1].set_xlabel("Number of nodes")
    axes[1].set_title('Haussdorff per cost function')
    axes[1].set_ylim(bottom=0)

x_tick_loc = [x + 1*shift for x in abscissa_pos]
axes[0].set_xticks(x_tick_loc, abscissa_pos)
axes[1].set_xticks(x_tick_loc, abscissa_pos)

legend_elements = [
    Line2D([0], [0], color='k', lw=0.01, marker=markers[0], linestyle=linestyles[0], label=cost_func_keys[0]),
    Line2D([0], [0], color='k', lw=0.01, marker=markers[1], linestyle=linestyles[1], label=cost_func_keys[1]),
    Line2D([0], [0], color='darkorange', lw=3, label="Exp sim geo graphs"),
]               
fig.suptitle("Comparison")
fig.subplots_adjust(bottom=0.17)
fig.legend(handles=legend_elements, loc="lower center", ncols=len(legend_elements))

## References

<a id="Izenman2008">[Izenman2008]</a>
Izenman Alan J. (2008). Introduction to Random-Matrix Theory [asc.ohio-state.edu]

<a id="Ryan2023">[Ryan2023]</a>
Sean Ryan and Rebecca Killick. Detecting Changes in Covariance via Random Matrix Theory. Technometrics, 65(4):480–491, October 2023. Publisher: Taylor & Francis